#### Imports

In [1]:
import time
from openai import OpenAI
from Assistant import AssistantOpenAI
import logging
import os
import json
import gymnasium as gym
import random
import matplotlib.pyplot as plt
import os
import time
from git import Repo

import numpy as np
#from collections import deque, namedtuple

# For visualization
import gymnasium.wrappers.record_video as record_video

#### Parameters

In [2]:
num_episodes = 200
ENV_NAME = 'LunarLander-v2'
ARCLABKEY_OPENAI = "sk-proj-DvHDR3hpgbm2r3kCA9jKT3BlbkFJL57ABXkfaWAIYKBxdhM6"
ARCLABKEY_OPENAI = "sk-proj-GvaDXazpibWA2M1I5Pu2T3BlbkFJxDuKlr9AcoVG98ctJZ7Q"

#### Function calling.

Función encargada de almacenar el código generado mediante la opción de function calling del asistente de OpenAI.

In [3]:
def store_code_in_file(reasoning, code, filename,):
    """ Store code in a file
    
    Args:
        reasoning: str: Pre-coding reasoning on log analysis and the influence of actions on observations and steps to follow to achieve a successful landing.
        code: str: code to store. Use the reasoning parameter to generate this code.
        filename: str: filename to store code in
    
    Returns:
        None
    """
    print(f"Model reasoning: {reasoning}")
    with open(filename, 'w') as f:
        f.write(code)
        
store_code_in_file_schema = {
    "name": "store_code_in_file",
    "description": "Store code in a file",
    "parameters": {
        "type": "object",
        "properties": {
            "reasoning": {
                "type": "string",
                "description": "Pre-coding deep reasoning on log analysis and the influence of actions on observations and steps to follow to achieve a successful landing. It is an expnasion but not the code itself."
            },
            "code": {
                "type": "string",
                "description": "The Python code to store. Use the reasoning parameter to generate this code."
            },
            "filename": {
                "type": "string",
                "description": "The filename to store the code in."
            }
        },
        "required": ["code", "filename"]
    }
}

TOOLS = [{"type": "function", "function": store_code_in_file_schema}]

#### Entorno Lunnar Lander

In [4]:
env = gym.make('LunarLander-v2')
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space.n)

State shape:  (8,)
Number of actions:  4


#### Código inicial

In [5]:
agent_initial_code = """
import random

def act(observation):
    '''
    The function that codifies the action to be taken in each instant of time.

    Args:
        observation (numpy.array):
            "description": "The state of the environment after the action is taken.",
            "positions": {  
                "0": "X position",
                "1": "Y position",
                "2": "X velocity",
                "3": "Y velocity",
                "4": "Angle",
                "5": "Angular velocity",
                "6": "Left contact sensor",
                "7": "Right contact sensor"
            },
            "min_values": [-1.5, -1.5, -5.0, -5.0, -3.14, -5.0, 0, 0],
            "max_values": [1.5, 1.5, 5.0, 5.0, 3.14, 5.0, 1, 1]

    Returns:
        Integer  : The action to be taken.
        "options": {
                '0' : "Switch off engines",
                '1' : "Push left engine",
                '2' : "Push both engines (upwards)",
                '3' : "Push right engine"
            }
    '''
    return random.randint(0, 3)
"""

#### Mapeador de logs a JSON

In [6]:
log_description = {
    "description": "Log data for each step of the spacecraft landing environment.",
    "landing attempt": {
        "type": "integer",
        "description": "The episode number."
    },
    "logs": {
        "time": {
            "type": "integer",
            "description": "The instant within the landing attempt where the current log is taken."
        },
        "action": {
            "type": "integer",
            "description": "The possible actions taken.",
            "options": {
                '0' : "Switch off engines",
                '1' : "Push left engine",
                '2' : "Push both engines (upwards)",
                '3' : "Push right engine"
            }
        },  
        "current status": {
            "type": "array",
            "description": "The state of the environment after the action is taken.",
            "positions": {  
                "0": {
                    "name": "X position",
                    "description": "The horizontal position of the spacecraft relative to the landing zone."
                },
                "1": {
                    "name": "Y position",
                    "description": "The vertical position of the spacecraft relative to the landing zone."
                },
                "2": {
                    "name": "X velocity",
                    "description": "The horizontal velocity of the spacecraft."
                },
                "3": {
                    "name": "Y velocity",
                    "description": "The vertical velocity of the spacecraft."
                },
                "4": {
                    "name": "Angle",
                    "description": "The angle of the spacecraft relative to the vertical (left negative, right positive)."
                },
                "5": {
                    "name": "Angular velocity",
                    "description": "The rate of change of the angle of the spacecraft."
                },
                "6": {
                    "name": "Left contact sensor with landing zone",
                    "description": "Indicates whether the left side of the spacecraft is in contact with the landing zone."
                },
                "7": {
                    "name": "Right contact sensor with landing zone",
                    "description": "Indicates whether the right side of the spacecraft is in contact with the landing zone."
                }
            },
            "min_values": [-1.5, -1.5, -5.0, -5.0, -3.14, -5.0, 0, 0],
            "max_values": [1.5, 1.5, 5.0, 5.0, 3.14, 5.0, 1, 1],
            
        }, 
        "score": {
            "type": "number",
            "description": "The score received for the action."
        },  
        "completed": {
            "type": "boolean",
            "description": "Whether the landing event has ended (landing or accident)."
        }
    },
    "total score": {
        "type": "number",
        "description": "The total score received for the landing attempt."
    }
}

#### Esquema JSON MODE

In [7]:
json_shcema_response = {
    "reasoning": {
        "type": "string",
        "description": "Pre-coding deep reasoning on log analysis and the influence of actions on observations and steps to follow to achieve a successful landing. It is an expnasion but not the code itself."
    },
    "code": {
        "type": "string",
        "description": "The Python code to store. Use the reasoning parameter to generate this code."
    },
    "filename": {
        "type": "string",
        "description": "The filename to store the code in (Action.py)."
    }
}

#### LOGS

In [8]:
import json

def format_step_data(instant, action, next_state, reward, terminated):
    """ Format the step data into a JSON string
    
    Args:
        instant: int: the instant within the landing attempt where the current log is taken
        action: int: the action taken
        next_state: np.array: the next state of the environment
        reward: float: the reward received
        terminated: bool: whether the landing event has ended
        
    Returns:
        str: the step data formatted as a JSON string
    """
    # Convertir el array numpy a una lista
    next_state_list = next_state.tolist()

    # Redondear los elementos de la lista a 4 decimales
    next_state_list_rounded = [round(x, 3) for x in next_state_list]

    step_data = {
        'time': instant,
        'action' : int(action),
        'current status': next_state_list_rounded,  # Convert numpy array to list
        'score': round(reward, 3),
        #'completed': terminated,
        #'truncated': truncated
        #'info': info
    }

    # Convert the dictionary to a JSON string
    #step_data_json = json.dumps(step_data)

    return step_data


def format_episode_logs(logs, episode, total_score):
    """ Format the logs into a JSON string
    
    Args:
        logs: list: the logs for each step of the environment
        episode: int: the episode number
        
    Returns:
        str: the logs formatted as a JSON string
    """
    logs_data = {
        'landing attempt': episode,
        'logs': logs,
        'total score': round(total_score, 3)
    }

    # Convert the dictionary to a JSON string
    logs_json = json.dumps(logs_data)

    return logs_json

#### Commit changes.

In [9]:
def commit_changes(repo_path, commit_message):
    """ Commit changes to the repository.

    Args:
        repo_path (str): Path to the repository.
        commit_message (str): The commit message.
    """
    repo = Repo(repo_path)
    repo.git.add(update=True)
    repo.index.commit(commit_message)
    origin = repo.remote(name='origin')
    origin.push()

#### Código encargado de ejecutar los eventos en el entorno.

In [10]:
import Action

def lunar_lander(max_t=1000, n_games=1, display=False, seed=38, agent=None, recoder=False, video_filename='video/video.mp4'):
    """ Run the Lunar Lander environment
    
    Args:
        max_t: int: the maximum number of timesteps
        n_games: int: the number of games to play
        display: bool: whether to display the environment
        seed: int: the seed for the environment
        agent: object: the agent to use
        recoder: bool: whether to record the video
        video_filename: str: the filename for the video
        
    Returns:
        str: the logs for the landing attempts
    """
    
    # Incluimos la opción del entorno gráfico y la de grabación.
    if display:
        env = gym.make(ENV_NAME, render_mode='human')
        if recoder:
            env = gym.make(ENV_NAME, render_mode='rgb_array')
            env.reset() 
            video_recorder = record_video.RecordVideo(env, video_filename)
    else:
        env = gym.make(ENV_NAME)

    # Bucle principal de ejecución de los episodios.
    logs = []
    for episode in range(1, n_games+1):
        # Revisamos que la semilla sea un número entero o una lista de semillas.
        if isinstance(seed, list):
            semilla = seed[episode-1]
        else:
            semilla = seed
        print(f"Semilla {semilla}")    
        state = env.reset(seed=semilla) # Set a seed for the environment
        state = state[0] # Eliminamos el diccionario vacio y dejamos unicamente el estado de 8 elementos.
        score = 0
        instant = 0
        episode_actions = []
        if recoder: # En caso de que se quiera grabar el video, se inicia el grabador.
            video_recorder.start_video_recorder()
            
        for i in range(max_t): # Bucle de ejecución de los instantes de tiempo.
            
            # Seleccionamos el método de elegir la acción (agente exitoso o método de decisión del asistente)
            if agent: 
                action = agent.act(state)
            else:
                action = Action.act(state)
            
            # Avanzamos un instante de tiempo en el entorno en función de si se quiere grabar el video o no.
            if recoder:
                next_state, reward, terminated, truncated, info = video_recorder.step(action)
            else:
                next_state, reward, terminated, truncated, info = env.step(action)
            
            # Formateamos los datos del instante de tiempo y los alcenamos.
            json_logs = format_step_data(instant, action, next_state, reward, terminated)
            score += reward
            instant += 1
            if len(episode_actions) < 20 or len(episode_actions) % 2 == 0 or terminated:
                logs.append(json_logs)
            #logs.append(json_logs)
            episode_actions.append(action)
            state = next_state
            if terminated: # Condición de salida del bucle, si el episodio ha terminado.
                break
        json_episode_logs = format_episode_logs(logs, episode, score) 
        
        print(f"Número de instantes: {instant+1}. Tamaño de logs: {len(logs)}")
        print('episode ', episode, 'score %.3f' % float(score), 'avg score %.3f' % (float(score) / instant))
        
    if recoder:
        video_recorder.close()
    else:
        env.close()
        
    return json_episode_logs

#### Logs del código inicial

In [11]:
# Genera una lista de n números enteros aleatorios entre 0 y 1000.
seeds = [random.randint(0, 1000) for i in range(4)]
initial_logs = lunar_lander(n_games=4, seed=seeds, display=True)
initial_logs

Semilla 626
Número de instantes: 414. Tamaño de logs: 217
episode  1 score 237.296 avg score 0.575
Semilla 411
Número de instantes: 175. Tamaño de logs: 315
episode  2 score -491.257 avg score -2.823
Semilla 217
Número de instantes: 147. Tamaño de logs: 399
episode  3 score -549.970 avg score -3.767
Semilla 533
Número de instantes: 247. Tamaño de logs: 533
episode  4 score -116.103 avg score -0.472


'{"landing attempt": 4, "logs": [{"time": 0, "action": 3, "current status": [-0.011, 1.431, -0.528, 0.425, 0.01, 0.085, 0.0, 0.0], "score": 0.977}, {"time": 1, "action": 3, "current status": [-0.016, 1.44, -0.517, 0.398, 0.012, 0.039, 0.0, 0.0], "score": 1.409}, {"time": 2, "action": 3, "current status": [-0.021, 1.448, -0.505, 0.371, 0.012, -0.009, 0.0, 0.0], "score": 1.735}, {"time": 3, "action": 3, "current status": [-0.026, 1.456, -0.494, 0.344, 0.009, -0.052, 0.0, 0.0], "score": 1.9}, {"time": 4, "action": 3, "current status": [-0.031, 1.463, -0.485, 0.318, 0.005, -0.087, 0.0, 0.0], "score": 1.909}, {"time": 5, "action": 3, "current status": [-0.035, 1.469, -0.478, 0.291, -0.001, -0.118, 0.0, 0.0], "score": 1.756}, {"time": 6, "action": 3, "current status": [-0.04, 1.475, -0.469, 0.265, -0.009, -0.155, 0.0, 0.0], "score": 0.703}, {"time": 7, "action": 3, "current status": [-0.045, 1.481, -0.461, 0.238, -0.018, -0.186, 0.0, 0.0], "score": 0.474}, {"time": 8, "action": 3, "current s

#### Logs exitosos
Utilizar otras semillas para que no memorice el entorno

In [12]:
import DuelingDQN

agent = DuelingDQN.Agent(num_observaciones=8, num_acciones=4, red_modelo=DuelingDQN.DuelingQNetwork, seed=0)
# Cargamos los pesos del agente entrenado.
agent.load_weights('checkpoint_Dueling.pth')

seeds = [130, 412]

success_logs = lunar_lander(n_games=len(seeds), display=True, seed=seeds, agent=agent)
success_logs

KeyboardInterrupt: 

#### Bucle iterativo

Sección principal del código encargada de conectar con el asistente de la API de OpenAI e iterar en la generación de código nuevo a partir de los registros del generado previamente.

In [ ]:
import traceback, importlib
import Action

Action = importlib.reload(Action)

def create_and_run_llm_loop(Incial_msg, logger, model="gpt-3.5-turbo", num_iterations=10):
    
    agente = AssistantOpenAI(ARCLABKEY_OPENAI)

    # Crea un asistente
    asistente = agente.create_assistant(model=model, description=DESCRIPTION, instructions=INSTRUCTIONS, name=NAME, tools=TOOLS)
    
    # Crea un hilo
    hilo = agente.create_thread()
    
    # Añade un mensaje inicial al hilo.
    msg = agente.add_message(hilo.id, role="user", content=Incial_msg)

    # Bucle de aprendizaje del asistente.
    for i in range(num_iterations):
        
        logger.info(f"Iteration: {i+1}")
        compiled = False

        # Si en la presente iteración no se ha compilado el código, se pide al asistente corregir los errores antes de iterar.
        while not compiled:
            # Ejecuta el hilo
            ejecucion = agente.run(hilo.id, asistente.id, tool_choice='required')
            response = agente.get_run(ejecucion.id, hilo.id)

            # Esperamos a que la respuesta esté lista.
            while response.status not in ["completed", "failed", "requires_action"]:
                logger.info(f"Status: {ejecucion.status}")
                response = agente.get_run(ejecucion.id, hilo.id)
                time.sleep(20)

            # Descomponemos los elementos de la respuesta.
            logger.info(f"Status: {response.status}")
            if response.status == "completed":
                return response
            tool_call = response.required_action.submit_tool_outputs.tool_calls
            print(f"Tool call: {tool_call}")

            # Convierte el string a un diccionario
            code_dict = json.loads(tool_call[0].function.arguments)
            logger.info(f"Arguments: {code_dict}")
            
            # Obtén el código Python de la llamada a la herramienta
            code = code_dict["code"]
            filename = code_dict["filename"]  
            reasoning = code_dict["reasoning"]

            logger.info(f"\nRazonamiento:\n{reasoning}")
            logger.info(f"\nCodigo generado:\n{code}")

            # Ejecuta el código Python
            try:
                store_code_in_file(reasoning, code, filename)
                time.sleep(1) # Pequeño retraso para que el sistema operativo pueda reflejar los cambios en el archivo
                
                # Hacemos commit de los cambios en el repositorio para analizar las modificaciones del agente.
                commit_changes(r"C:\Users\adria\Repos\TFM__LLM_landing_self-refinement", f"4o Setpback. Iteración {i+1}.") 
                
                importlib.reload(Action) # Recargamos el módulo de acciones para que se actualice con las modificaciones del agente.

                # Ejecutamos el código generado.
                logs = lunar_lander(n_games=1, display=True, recoder=True, video_filename=f"video/iteration_{i+1}.mp4")
                
                # Devolvemos la respuesta al asistente.
                for call in tool_call:
                    agente.devolver_respuesta(response.id, hilo.id, tool_outputs=[{"tool_call_id": call.id, "output": "Run successful."}])            
                compiled = True
                logger.info(f"Compilación exitosa.")
                
                # Esperamos a que el agente esté listo para recibir mensajes y le añadimos el resultado de la iteración.
                while response.status not in ["completed", "failed", "expired"]:
                    logger.info(f"Status: {response.status}")
                    response = agente.get_run(response.id, hilo.id)
                    time.sleep(20)
                
                msg = f"""These are the logs generated by your last code: {logs}. Analyze the performance of the spacecraft and how it differs from the desired result. Be guided by the scoring system. Identify the cause of errors in your code and modify it without fear of making major changes. Think deeply about the priorities of your code and how to order and combine them correctly to achieve success. Remember that the code is a reflection of your reasoning and the way you approach the problem."""
                logger.info(msg)    
                agente.add_message(hilo.id, role="user", content=msg)
            
            # Alimentamos el asistente con el error generado en la ejecución del código.    
            except Exception as e:
                logger.exception("Error: %s", e)
                error_trace = traceback.format_exc()
                for call in tool_call:
                    agente.devolver_respuesta(response.id, hilo.id, tool_outputs=[{"tool_call_id": call.id, "output": "ERROR."}]) 
                logger.error(f"Error: {e}.")
                while response.status not in ["completed", "failed", "expired"]:
                    logger.info(f"Status: {response.status}")
                    response = agente.get_run(ejecucion.id, hilo.id)
                    time.sleep(30)
                msg = f"The code generated has an error. Please, try again. Error: {e}. Trace: {error_trace}"   
                logger.error(msg)   
                agente.add_message(hilo.id, role="assistant", content=msg)
    
    agente.mostrar_mensajes(hilo.id)  
    vaciar_agente(agente)
    
    logger.info("\nEjecución finalizada.\n\n")       

    return response


def vaciar_agente(agente):
    for assistant in agente.assistants:
        agente.delete_assistant(assistant)

    for thread in agente.threads:
        agente.delete_thread(thread)
        
    print("Asistente vaciado.")


#### Logger

In [ ]:
def configura_log(nombre_archivo):
    """ 
    Configura el logger para que guarde los logs en un archivo y los muestre en la consola.
    
    Args:
        nombre_archivo: str Nombre del archivo donde se guardarán los logs.
        
    Returns:
        logger: logging.Logger Objeto logger configurado.
    """
    # Crear la carpeta logs si no existe
    if not os.path.exists('logs'):
        os.makedirs('logs')
    # Configura el logger
    logging.basicConfig(filename=f'logs/{nombre_archivo}', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logger = logging.getLogger()
    
    # Añade un StreamHandler para mostrar los logs en la consola
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)
    
    return logger

#### Ejecución con el asistente

Prompt inicial y mensajes del sistema para el asistente.

In [ ]:
DESCRIPTION = "You are an expert programer in Pyhton. Your specialty is to generate the code responsible for making decisions about actions to be taken in various spacecraft landing environments.The objective is to land the spacecraft within a target zone in the shortest possible time and very gently. A scoring system is used to evaluate the landings, which must be maximized."
INSTRUCTIONS = f"""Your task is:
You have two modes of action depending on whether the use of function call is required. Reason deeply about what you are asked to do or code the requested task.
1. Analyze and reason about the logs received in the last landing attempts.
2. Your goal is to be able to make the correct decision based on what you have learned from the results of previous iterations. You must code the decision making based on your reasoning in a Python function.
3. IMPORTANT. Use the following tips in your reasoning to achieve a successful landing:
    - First you have to stabilize the falling ship (angle and location), keep falling under control and then land gently at the end.
    - It is mandatory to use all the elements of the array of observations received by parameter in your code when deciding what action to take at any given moment. Both position and velocities must be taken into account to know how the ship is doing and towards which states it is heading. All of these must be considered to achieve stability.
    - Learn how actions taken affect the future states of the spacecraft in the logs of past events so that you can take this into account when developing code to reach the landing zone.
    - The landing zone is in the central area of the x-axis.
    - Carefully choose the actions involving the lateral engines according to the rotation of the spacecraft.
4. This is the schema of the landing logs: {log_description}.
5. You should analyze the performance that appear in the logs of the code you have generated. You should improve the code generated in the 'act' function in the last iteration without fear of making major changes, seeking to maximize the score received and generate a higher quality code.
6. Save the code of the act function in the file 'Action.py' using store_code_in_file function tool.
"""
# 6. Improve your results and correct also any programming error you may have generated in your last code if they exist.
NAME = "Spacecraft Landing Master"

initial_msg = f"This is the record of an example of a successful landing in this environment, but under other conditions: {success_logs}. You have to be able to learn from it to land successfully with any other conditions. This is the code of the initial function: {agent_initial_code} and these are the execution logs of one landing attempt: {initial_logs}.Take a deep breath and reason step-by-step. After reasoning analyze the results, learn and make better code."

logger = configura_log('Spacecraft_4o_betterprompt.log')
response = create_and_run_llm_loop(initial_msg, logger, model="gpt-4o", num_iterations=5)

print(response)

2024-07-02 00:37:43,931 - INFO - HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
2024-07-02 00:37:43,931 - INFO - HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
2024-07-02 00:37:44,122 - INFO - HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"
2024-07-02 00:37:44,122 - INFO - HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"
2024-07-02 00:37:44,385 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_QoB6MCbRbYhGkgNXtosUNZOa/messages "HTTP/1.1 200 OK"
2024-07-02 00:37:44,385 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_QoB6MCbRbYhGkgNXtosUNZOa/messages "HTTP/1.1 200 OK"
2024-07-02 00:37:44,532 - INFO - Iteration: 1
2024-07-02 00:37:44,532 - INFO - Iteration: 1
2024-07-02 00:37:44,929 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_QoB6MCbRbYhGkgNXtosUNZOa/runs "HTTP/1.1 200 OK"
2024-07-02 00:37:44,929 - INFO - HTTP Request: POST htt

Tool call: [RequiredActionFunctionToolCall(id='call_aFYAf5toEx9JxbWztDqncIya', function=Function(arguments='{"code":"import numpy as np\\n\\ndef act(observation):\\n    \'\'\'\\n    The function that codifies the action to be taken in each instant of time.\\n\\n    Args:\\n        observation (numpy.array):\\n            \\"description\\": \\"The state of the environment after the action is taken.\\",\\n            \\"positions\\": {  \\n                \\"0\\": \\"X position\\",\\n                \\"1\\": \\"Y position\\",\\n                \\"2\\": \\"X velocity\\",\\n                \\"3\\": \\"Y velocity\\",\\n                \\"4\\": \\"Angle\\",\\n                \\"5\\": \\"Angular velocity\\",\\n                \\"6\\": \\"Left contact sensor\\",\\n                \\"7\\": \\"Right contact sensor\\"\\n            },\\n            \\"min_values\\": [-1.5, -1.5, -5.0, -5.0, -3.14, -5.0, 0, 0],\\n            \\"max_values\\": [1.5, 1.5, 5.0, 5.0, 3.14, 5.0, 1, 1]\\n\\n    Returns:

c:\Users\adria\anaconda3\Lib\site-packages\gymnasium\wrappers\record_video.py:94: UserWarning: WARN: Overwriting existing videos at c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Semilla 38
Moviepy - Building video c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4\rl-video-episode-0.mp4.
Moviepy - Writing video c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4\rl-video-episode-0.mp4



2024-07-02 00:38:30,678 - ERROR - Error: Missing required arguments; Expected either ('thread_id' and 'tool_outputs') or ('thread_id', 'stream' and 'tool_outputs') arguments to be given
Traceback (most recent call last):
  File "C:\Users\adria\AppData\Local\Temp\ipykernel_19840\4102630422.py", line 69, in create_and_run_llm_loop
    agente.devolver_respuesta(response.id, hilo.id, tool_outputs=[{"tool_call_id": call.id, "output": "Run successful."}])
  File "c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\Assistant.py", line 171, in devolver_respuesta
    response = self.client.beta.threads.runs.submit_tool_outputs(run_id, thread_id=thread_id, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\adria\anaconda3\Lib\site-packages\openai\_utils\_utils.py", line 276, in wrapper
    raise TypeError(msg)
TypeError: Missing required arguments; Expected either ('thread_id' and 'tool_outputs') or ('thread_id', '

Moviepy - Done !
Moviepy - video ready c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4\rl-video-episode-0.mp4
Número de instantes: 73. Tamaño de logs: 47
episode  1 score -612.950 avg score -8.513


TypeError: Missing required arguments; Expected either ('thread_id' and 'tool_outputs') or ('thread_id', 'stream' and 'tool_outputs') arguments to be given

#### Prueba de cambio grande en el prompt.

In [14]:
DESCRIPTION = "You are an expert spacecraft landing agent. Your specialty is to take the appropriate action at each instant of time based on the environment and state of the spacecraft. To achieve this, you put your knowledge base and the knowledge you acquire by analyzing each execution into a python function in charge of directing the landing."
INSTRUCTIONS = f"""To complete the task you must follow the following steps and indications:
1. Analyze and reason about the records received in the last landing attempts to learn how the spacecraft behaves in the environment. Keep in mind that conditions may vary.
2. To measure how good a landing is, a scoring system that appears in the logs is used, your role is to maximize it. To do so, keep the following in mind:
    - Increases/decreases the closer/further the spacecraft is from the landing area (both axes).
    - Increased/decreased the slower/faster the spacecraft is moving.
    - Decreased the more the spacecraft is tilted.
    - Decreased by 0.03 points each frame a side engine is firing and 0.3 points for the center engine.
    - Receive 100 points for a successful landing and lose them for crashing.
3. IMPORTANT. Use the following tips in your reasoning to achieve a successful landing:
    - First you have to stabilize the falling ship (speed, angle and location), keep falling under control and then land gently at the end.
    - Use all the elements of the vector of observations, all are relevant to make the right decision.
    - Pay close attention to successful events.
    - Environment and ship conditions may change but your code must be effective for all cases.
    - Find a balance in your landing policy to maximize your score (must exceed 200 points).
    - If you are rotating to the left (negative values), you should fire the left engine and vice versa. But always prioritizing a controlled speed with the main engine.
4. This is the schema of the landing logs: {log_description}.
5. You should analyze the performance that appear in the logs of the code you have generated. You should improve the code generated in the 'act' function in the last iteration without fear of making major changes, seeking to maximize the score received and generate a higher quality code.
6. Save the code of the act function in the file 'Action.py' using store_code_in_file function tool.
"""

# 6. Improve your results and correct also any programming error you may have generated in your last code if they exist.
NAME = "Spacecraft Landing Master"

initial_msg = f"This is the record of an example of a successful landing in this environment, but under other conditions: {success_logs}. This is the code of the initial function: {agent_initial_code} and these are the execution logs of one landing attempt: {initial_logs}. What does each value and each change in the observations mean? What effects does each action have on the current state observations? What are the steps to follow to achieve a successful landing?. Take a deep breath and reason step-by-step. After reasoning analyze the results, learn and make better code."

logger = configura_log('Spacecraft_4o_Stepback.log')
response = create_and_run_llm_loop(initial_msg, logger, model="gpt-4o", num_iterations=5)

print(response)

2024-07-02 01:28:50,965 - INFO - HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
2024-07-02 01:28:51,143 - INFO - HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"
2024-07-02 01:28:51,438 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_L7ucr9hmSMXelhzWoyVcJn7v/messages "HTTP/1.1 200 OK"
2024-07-02 01:28:51,571 - INFO - Iteration: 1
2024-07-02 01:28:51,986 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_L7ucr9hmSMXelhzWoyVcJn7v/runs "HTTP/1.1 200 OK"
2024-07-02 01:28:52,391 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_L7ucr9hmSMXelhzWoyVcJn7v/runs/run_Pt7gqDwYum9j3Ux59bApeiz7 "HTTP/1.1 200 OK"
2024-07-02 01:28:52,394 - INFO - Status: queued
2024-07-02 01:28:52,595 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_L7ucr9hmSMXelhzWoyVcJn7v/runs/run_Pt7gqDwYum9j3Ux59bApeiz7 "HTTP/1.1 200 OK"
2024-07-02 01:29:12,599 - INFO - Status: queued
2024-07-02 01:29:12,854

Tool call: [RequiredActionFunctionToolCall(id='call_09KGpk8yWlzAUJfv4sZ0RvbY', function=Function(arguments='{"reasoning": "### Analysis and Reasoning\\n\\n#### Observations and Actions:\\n1. **X and Y Position (`obs[0]`, `obs[1]`)**: These are the horizontal and vertical positions of the spacecraft. The goal is to bring the X position to 0 (centered on the landing pad) and the Y position to 0 (landed on the pad).\\n2. **X and Y Velocity (`obs[2]`, `obs[3]`)**: These are the velocities in the X and Y directions. We aim to reduce these to near zero for a soft landing.\\n3. **Angle (`obs[4]`)**: This is the tilt angle of the spacecraft. We aim to keep this at 0 (vertical) to avoid tipping over.\\n4. **Angular Velocity (`obs[5]`)**: This is the rate at which the angle is changing. This should also be minimized to ensure the spaceship is stable.\\n5. **Left and Right Contact Sensors (`obs[6]`, `obs[7]`)**: These indicate if the left or right side of the spacecraft is in contact with the gro

c:\Users\adria\anaconda3\Lib\site-packages\gymnasium\wrappers\record_video.py:94: UserWarning: WARN: Overwriting existing videos at c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Semilla 38
Moviepy - Building video c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4\rl-video-episode-0.mp4.
Moviepy - Writing video c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4\rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4\rl-video-episode-0.mp4
Número de instantes: 145. Tamaño de logs: 83
episode  1 score -23.203 avg score -0.161


...

Moviepy - Done !
Moviepy - video ready c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_5.mp4\rl-video-episode-0.mp4
Número de instantes: 86. Tamaño de logs: 53
episode  1 score -116.692 avg score -1.373


2024-07-02 01:35:19,698 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_L7ucr9hmSMXelhzWoyVcJn7v/runs/run_nOjPMiKC59rz6dTUyDqrDpkW/submit_tool_outputs "HTTP/1.1 200 OK"
2024-07-02 01:35:19,701 - INFO - Compilación exitosa.
2024-07-02 01:35:19,702 - INFO - Status: requires_action
2024-07-02 01:35:19,905 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_L7ucr9hmSMXelhzWoyVcJn7v/runs/run_nOjPMiKC59rz6dTUyDqrDpkW "HTTP/1.1 200 OK"
2024-07-02 01:35:39,910 - INFO - Status: in_progress
2024-07-02 01:35:40,156 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_L7ucr9hmSMXelhzWoyVcJn7v/runs/run_nOjPMiKC59rz6dTUyDqrDpkW "HTTP/1.1 200 OK"
2024-07-02 01:36:00,161 - INFO - These are the logs generated by your last code: {"landing attempt": 1, "logs": [{"time": 0, "action": 0, "current status": [0.008, 1.417, 0.38, 0.131, -0.009, -0.085, 0.0, 0.0], "score": 0.232}, {"time": 1, "action": 0, "current status": [0.011, 1.42, 0.38, 0.104, -0.013, -0

user: [TextContentBlock(text=Text(annotations=[], value='These are the logs generated by your last code: {"landing attempt": 1, "logs": [{"time": 0, "action": 0, "current status": [0.008, 1.417, 0.38, 0.131, -0.009, -0.085, 0.0, 0.0], "score": 0.232}, {"time": 1, "action": 0, "current status": [0.011, 1.42, 0.38, 0.104, -0.013, -0.085, 0.0, 0.0], "score": 0.121}, {"time": 2, "action": 0, "current status": [0.015, 1.421, 0.38, 0.077, -0.017, -0.085, 0.0, 0.0], "score": 0.013}, {"time": 3, "action": 0, "current status": [0.019, 1.423, 0.38, 0.051, -0.021, -0.085, 0.0, 0.0], "score": -0.104}, {"time": 4, "action": 0, "current status": [0.023, 1.423, 0.38, 0.024, -0.026, -0.085, 0.0, 0.0], "score": -0.226}, {"time": 5, "action": 0, "current status": [0.026, 1.423, 0.38, -0.003, -0.03, -0.085, 0.0, 0.0], "score": -0.353}, {"time": 6, "action": 0, "current status": [0.03, 1.422, 0.38, -0.029, -0.034, -0.085, 0.0, 0.0], "score": -0.481}, {"time": 7, "action": 0, "current status": [0.034, 1.42

2024-07-02 01:36:01,593 - INFO - HTTP Request: DELETE https://api.openai.com/v1/assistants/asst_oz37cI4Jj4423yR963RqZPpk "HTTP/1.1 200 OK"
2024-07-02 01:36:01,926 - INFO - HTTP Request: DELETE https://api.openai.com/v1/threads/thread_L7ucr9hmSMXelhzWoyVcJn7v "HTTP/1.1 200 OK"
2024-07-02 01:36:01,927 - INFO - 
Ejecución finalizada.




Asistente vaciado.
Run(id='run_nOjPMiKC59rz6dTUyDqrDpkW', assistant_id='asst_oz37cI4Jj4423yR963RqZPpk', cancelled_at=None, completed_at=1719876926, created_at=1719876875, expires_at=None, failed_at=None, incomplete_details=None, instructions="To complete the task you must follow the following steps and indications:\n1. Analyze and reason about the records received in the last landing attempts to learn how the spacecraft behaves in the environment. Keep in mind that conditions may vary.\n2. To measure how good a landing is, a scoring system that appears in the logs is used, your role is to maximize it. To do so, keep the following in mind:\n    - Increases/decreases the closer/further the spacecraft is from the landing area (both axes).\n    - Increased/decreased the slower/faster the spacecraft is moving.\n    - Decreased the more the spacecraft is tilted.\n    - Decreased by 0.03 points each frame a side engine is firing and 0.3 points for the center engine.\n    - Receive 100 points 

In [14]:
DESCRIPTION = "You are an expert spacecraft landing agent. Your specialty is to take the appropriate action at each instant of time based on the environment and state of the spacecraft. To achieve this, you put your knowledge base and the knowledge you acquire by analyzing each execution into a python function in charge of directing the landing."
INSTRUCTIONS = f"""To complete the task you must follow the following steps and indications:
1. Analyze and reason about the records received in the last landing attempts to learn how the spacecraft behaves in the environment. Keep in mind that conditions may vary.
2. To measure how good a landing is, a scoring system that appears in the logs is used, your role is to maximize it. To do so, keep the following in mind:
    - Increases/decreases the closer/further the spacecraft is from the landing area (both axes).
    - Increased/decreased the slower/faster the spacecraft is moving.
    - Decreased the more the spacecraft is tilted.
    - Decreased by 0.03 points each frame a side engine is firing and 0.3 points for the center engine.
    - Receive 100 points for a successful landing and lose them for crashing.
    - A succesful landing must exceed 200 points.
3. This is the log schema:\n {log_description}
4. Add your decision code to the 'act' method and locate errors in it if the landing is unsuccessful.
5. Save the code of the act function in the file 'Action.py' using store_code_in_file function.
"""

# 6. Improve your results and correct also any programming error you may have generated in your last code if they exist.
NAME = "Spacecraft Landing Master"

initial_msg = f"This is the record of an example of a successful landing in this environment, but under other conditions: {success_logs}. This is the code of the initial function: {agent_initial_code} and these are the execution logs of one landing attempt: {initial_logs}. What does each value and each change in the observations mean? What effects does each action have on the current state observations? What are the steps to follow to achieve a successful landing?. Take a deep breath and reason step-by-step. After reasoning analyze the results, learn and make better code."

logger = configura_log('Spacecraft_4o_Stepback.log')
response = create_and_run_llm_loop(initial_msg, logger, model="gpt-4o", num_iterations=5)

print(response)

2024-07-02 20:13:38,487 - INFO - HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
2024-07-02 20:13:38,682 - INFO - HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"
2024-07-02 20:13:38,976 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_ZDDzGedNEV4UQxGCR92T9JFg/messages "HTTP/1.1 200 OK"
2024-07-02 20:13:39,134 - INFO - Iteration: 1
2024-07-02 20:13:39,768 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_ZDDzGedNEV4UQxGCR92T9JFg/runs "HTTP/1.1 200 OK"
2024-07-02 20:13:40,001 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_ZDDzGedNEV4UQxGCR92T9JFg/runs/run_Df58B0YObxg72VbXW3CpIngt "HTTP/1.1 200 OK"
2024-07-02 20:13:40,003 - INFO - Status: queued
2024-07-02 20:13:40,209 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_ZDDzGedNEV4UQxGCR92T9JFg/runs/run_Df58B0YObxg72VbXW3CpIngt "HTTP/1.1 200 OK"
2024-07-02 20:14:00,212 - INFO - Status: queued
2024-07-02 20:14:00,474

Tool call: [RequiredActionFunctionToolCall(id='call_OImymi36vVR9f8le1Dvbn3N8', function=Function(arguments='{"reasoning":"Analyzing the successful and unsuccessful landing attempts, the spacecraft\'s behavior can be broken down into several key observations and corresponding actions:\\n\\n1. **X Position and Velocity**: Ideally, keep the spacecraft near the center (X position ≈ 0) with minimal horizontal velocity (X velocity ≈ 0). Pushing the left engine (Action 1) moves the spacecraft right, and pushing the right engine (Action 3) moves it left. Pushing both engines (Action 2) slightly affects the horizontal velocity but mainly affects vertical dynamics.\\n2. **Y Position and Velocity**: The goal is to reduce the Y velocity (downward speed) to zero as the spacecraft approaches the ground. Pushing both engines (Action 2) reduces the Y velocity (slows descent), while inaction (Action 0) allows gravity to increase the descent speed.\\n3. **Angle and Angular Velocity**: Keep the spacecraf

c:\Users\adria\anaconda3\Lib\site-packages\gymnasium\wrappers\record_video.py:94: UserWarning: WARN: Overwriting existing videos at c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Semilla 38
Moviepy - Building video c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4\rl-video-episode-0.mp4.
Moviepy - Writing video c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4\rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4\rl-video-episode-0.mp4
Número de instantes: 77. Tamaño de logs: 49
episode  1 score -141.601 avg score -1.863


...

Semilla 38
Moviepy - Building video c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_4.mp4\rl-video-episode-0.mp4.
Moviepy - Writing video c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_4.mp4\rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_4.mp4\rl-video-episode-0.mp4
Número de instantes: 77. Tamaño de logs: 49
episode  1 score -139.587 avg score -1.837


2024-07-02 20:31:25,926 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_ZDDzGedNEV4UQxGCR92T9JFg/runs/run_Mbk3Bg1ezOMUtB3M2yjxtIIC/submit_tool_outputs "HTTP/1.1 200 OK"
2024-07-02 20:31:25,927 - INFO - Compilación exitosa.
2024-07-02 20:31:25,928 - INFO - Status: requires_action
2024-07-02 20:31:26,133 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_ZDDzGedNEV4UQxGCR92T9JFg/runs/run_Mbk3Bg1ezOMUtB3M2yjxtIIC "HTTP/1.1 200 OK"
2024-07-02 20:31:46,135 - INFO - Status: in_progress
2024-07-02 20:31:46,390 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_ZDDzGedNEV4UQxGCR92T9JFg/runs/run_Mbk3Bg1ezOMUtB3M2yjxtIIC "HTTP/1.1 200 OK"
2024-07-02 20:32:06,393 - INFO - These are the logs generated by your last code: {"landing attempt": 1, "logs": [{"time": 0, "action": 1, "current status": [0.007, 1.417, 0.37, 0.131, -0.007, -0.044, 0.0, 0.0], "score": 1.387}, {"time": 1, "action": 1, "current status": [0.011, 1.42, 0.362, 0.104, -0.007, -

Tool call: [RequiredActionFunctionToolCall(id='call_I9MbO74c2ydNkxlHZTE5XqWU', function=Function(arguments='{"reasoning":"The logs reveal persistent challenges: \\n1. **Excessive Engine Usage**: Continuous use of left and right engines leads to increased horizontal speed. \\n2. **Angle and Angular Velocity Control**: Stabilizing these parameters hasn\'t been sufficiently dynamic. \\n3. **Vertical Stability**: Over-correction with the center engine is notable, impacting descent stability. \\n4. **Inefficiency Near Touchdown**: Contact sensor engagement shows instability, indicating poor fine control near the ground.\\n\\n### Updated Strategy for Successful Landing:\\n1. **Primary Focus**: Prioritize dynamic angle and angular velocity stabilization. \\n2. **Secondary Focus**: Ensure vertical stability using center engines judiciously.\\n3. **Tertiary Focus**: Horizontal position corrections based on stable angle maintenance.\\n4. **Efficient Engine Use**: Minimize firing duration to cons

KeyError: 'code'

#### Prompt antiguo exitoso.

In [15]:
DESCRIPTION = "You are an expert programer in Pyhton. Your specialty is to generate the code responsible for making decisions about actions to be taken in various spacecraft landing environments.The objective is to land the spacecraft within a target zone in the shortest possible time and very gently. A scoring system is used to evaluate the landings, which must be maximized."
INSTRUCTIONS = f"""Your task is: 
    1. Analyze and reason about the results received in the last landing attempts. This is the schema of the landing logs: {log_description}.
    2. Your goal is to be able to make the appropriate decision based on the results of previous iterations. You must code the decision making based on your reasoning in a Python function.
    3. Use these tips to get a good landing:
        - First you have to stabilize the falling ship and then land gently.
        - Use all observations when deciding what action to take at any given moment. Both position and velocities must be taken into account to know how the ship is doing and towards which states it is heading. All of these must be considered to achieve stability.
        - Landing zone is in the central area of the x-axis.
        - Learn how actions taken affect the future states of the spacecraft in the logs of past events so that you can take this into account when developing code to reach the landing zone.
    4. Based on the logs you should improve the code generated in the 'act' function in the last iteration, seeking to maximize the score received and generate a higher quality code.
    5. Save the code of the act function in the file 'Action.py' using the tools given.
    6. Improve your results and correct any errors you may have generated in your last code if they exist."""

NAME = "Spacecraft Landing Master"

initial_msg = f"Take a deep breath and think step by step. This is the record of an example of a successful landing in this environment, but under other conditions: {success_logs}. This is the code of the initial function: {agent_initial_code} and these are the execution logs of one landing attempt: {initial_logs}. Analyze the results and improve the code."
logger = configura_log('Spacecraft_4o_exitosoAntiguo.log')
response = create_and_run_llm_loop(initial_msg, logger, model="gpt-4o", num_iterations=5)

print(response)

2024-07-03 17:34:11,146 - INFO - HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
2024-07-03 17:34:11,342 - INFO - HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"
2024-07-03 17:34:11,635 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_6wvsBNrPNjBxrOkvv9jtQGOv/messages "HTTP/1.1 200 OK"
2024-07-03 17:34:11,779 - INFO - Iteration: 1
2024-07-03 17:34:12,239 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_6wvsBNrPNjBxrOkvv9jtQGOv/runs "HTTP/1.1 200 OK"
2024-07-03 17:34:12,469 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_6wvsBNrPNjBxrOkvv9jtQGOv/runs/run_eS9Z979IPNxLpwlCs5S2JEtJ "HTTP/1.1 200 OK"
2024-07-03 17:34:12,477 - INFO - Status: queued
2024-07-03 17:34:12,708 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_6wvsBNrPNjBxrOkvv9jtQGOv/runs/run_eS9Z979IPNxLpwlCs5S2JEtJ "HTTP/1.1 200 OK"
2024-07-03 17:34:32,711 - INFO - Status: queued
2024-07-03 17:34:33,007

Tool call: [RequiredActionFunctionToolCall(id='call_6Kl599fRlbIZSXnqisbrKZ2R', function=Function(arguments='{"reasoning":"After analyzing the provided logs for both a successful and unsuccessful landing, the following strategies can be derived:\\n1. Stabilize the spacecraft\'s vertical speed and angle before attempting to land.\\n2. Keep the X position close to the center.\\n3. Use actions that counteract undesirable velocities and angles.\\n4. Ensure the craft\'s vertical descent is minimized, especially as it gets closer to ground.\\n\\nUsing these strategies, a proportional control approach can be implemented, which continually adjusts the actions based on the observed state, targeting zero horizontal and vertical speeds, and a stable angle. Minor adjustments should be made until the vessel is stably landed.\\n\\nHere is an improved version of the \'act\' function that uses proportional control logic to decide on the appropriate action based on real-time observations:\\n","code":"de

c:\Users\adria\anaconda3\Lib\site-packages\gymnasium\wrappers\record_video.py:94: UserWarning: WARN: Overwriting existing videos at c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Semilla 38
Moviepy - Building video c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4\rl-video-episode-0.mp4.
Moviepy - Writing video c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4\rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4\rl-video-episode-0.mp4
Número de instantes: 75. Tamaño de logs: 48
episode  1 score -634.961 avg score -8.581


...

Moviepy - Done !
Moviepy - video ready c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_5.mp4\rl-video-episode-0.mp4
Número de instantes: 94. Tamaño de logs: 57
episode  1 score -557.953 avg score -5.999


2024-07-03 17:40:40,964 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_6wvsBNrPNjBxrOkvv9jtQGOv/runs/run_80UymoumzihMvIMeNc56gOs5/submit_tool_outputs "HTTP/1.1 200 OK"
2024-07-03 17:40:40,966 - INFO - Compilación exitosa.
2024-07-03 17:40:40,967 - INFO - Status: requires_action
2024-07-03 17:40:41,188 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_6wvsBNrPNjBxrOkvv9jtQGOv/runs/run_80UymoumzihMvIMeNc56gOs5 "HTTP/1.1 200 OK"
2024-07-03 17:41:01,192 - INFO - Status: in_progress
2024-07-03 17:41:01,454 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_6wvsBNrPNjBxrOkvv9jtQGOv/runs/run_80UymoumzihMvIMeNc56gOs5 "HTTP/1.1 200 OK"
2024-07-03 17:41:21,457 - INFO - These are the logs generated by your last code: {"landing attempt": 1, "logs": [{"time": 0, "action": 0, "current status": [0.008, 1.417, 0.38, 0.131, -0.009, -0.085, 0.0, 0.0], "score": 0.232}, {"time": 1, "action": 0, "current status": [0.011, 1.42, 0.38, 0.104, -0.013, -0

user: [TextContentBlock(text=Text(annotations=[], value='These are the logs generated by your last code: {"landing attempt": 1, "logs": [{"time": 0, "action": 0, "current status": [0.008, 1.417, 0.38, 0.131, -0.009, -0.085, 0.0, 0.0], "score": 0.232}, {"time": 1, "action": 0, "current status": [0.011, 1.42, 0.38, 0.104, -0.013, -0.085, 0.0, 0.0], "score": 0.121}, {"time": 2, "action": 0, "current status": [0.015, 1.421, 0.38, 0.077, -0.017, -0.085, 0.0, 0.0], "score": 0.013}, {"time": 3, "action": 0, "current status": [0.019, 1.423, 0.38, 0.051, -0.021, -0.085, 0.0, 0.0], "score": -0.104}, {"time": 4, "action": 0, "current status": [0.023, 1.423, 0.38, 0.024, -0.026, -0.085, 0.0, 0.0], "score": -0.226}, {"time": 5, "action": 0, "current status": [0.026, 1.423, 0.38, -0.003, -0.03, -0.085, 0.0, 0.0], "score": -0.353}, {"time": 6, "action": 0, "current status": [0.03, 1.422, 0.38, -0.029, -0.034, -0.085, 0.0, 0.0], "score": -0.481}, {"time": 7, "action": 0, "current status": [0.034, 1.42

2024-07-03 17:41:22,476 - INFO - HTTP Request: DELETE https://api.openai.com/v1/assistants/asst_PkpU83LGyU1PPCgGtjUcUqx8 "HTTP/1.1 200 OK"
2024-07-03 17:41:22,692 - INFO - HTTP Request: DELETE https://api.openai.com/v1/threads/thread_6wvsBNrPNjBxrOkvv9jtQGOv "HTTP/1.1 200 OK"
2024-07-03 17:41:22,693 - INFO - 
Ejecución finalizada.




Asistente vaciado.
Run(id='run_80UymoumzihMvIMeNc56gOs5', assistant_id='asst_PkpU83LGyU1PPCgGtjUcUqx8', cancelled_at=None, completed_at=1720021245, created_at=1720021195, expires_at=None, failed_at=None, incomplete_details=None, instructions="Your task is: \n    1. Analyze and reason about the results received in the last landing attempts. This is the schema of the landing logs: {'description': 'Log data for each step of the spacecraft landing environment.', 'landing attempt': {'type': 'integer', 'description': 'The episode number.'}, 'logs': {'time': {'type': 'integer', 'description': 'The instant within the landing attempt where the current log is taken.'}, 'action': {'type': 'integer', 'description': 'The possible actions taken.', 'options': {'0': 'Switch off engines', '1': 'Push left engine', '2': 'Push both engines (upwards)', '3': 'Push right engine'}}, 'current status': {'type': 'array', 'description': 'The state of the environment after the action is taken.', 'positions': {'0':

#### Prompt zero shot.

In [15]:
DESCRIPTION = "You are an expert programer in Pyhton. Your specialty is to generate the code responsible for making decisions about actions to be taken in various spacecraft landing environments.The objective is to land the spacecraft within a target zone in the shortest possible time and very gently. A scoring system is used to evaluate the landings, which must be maximized."
INSTRUCTIONS = f"""Your task is: 
    1. Analyze and reason about the results received in the last landing attempts. This is the schema of the landing logs: {log_description}.
    2. Your goal is to be able to make the appropriate decision based on the results of previous iterations. You must code the decision making based on your reasoning in a Python function.
    3. Use these tips to get a good landing:
        - First you have to stabilize the falling ship and then land gently.
        - Use all observations when deciding what action to take at any given moment. Both position and velocities must be taken into account to know how the ship is doing and towards which states it is heading. All of these must be considered to achieve stability.
        - Landing zone is in the central area of the x-axis.
        - Learn how actions taken affect the future states of the spacecraft in the logs of past events so that you can take this into account when developing code to reach the landing zone.
    4. Based on the logs you should improve the code generated in the 'act' function in the last iteration, seeking to maximize the score received and generate a higher quality code.
    5. Save the code of the act function in the file 'Action.py' using the tools given.
    6. Improve your results and correct any errors you may have generated in your last code if they exist."""

NAME = "Spacecraft Landing Master"

initial_msg = f"Take a deep breath and think step by step. Analyze the results and improve the code."
logger = configura_log('Spacecraft_4o_ZeroShot.log')
response = create_and_run_llm_loop(initial_msg, logger, model="gpt-4o", num_iterations=5)

print(response)

2024-07-03 19:59:17,407 - INFO - HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
2024-07-03 19:59:17,633 - INFO - HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"
2024-07-03 19:59:17,853 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_oJWBt4hwO9PwfGOyZ8H3CfAK/messages "HTTP/1.1 200 OK"
2024-07-03 19:59:17,860 - INFO - Iteration: 1
2024-07-03 19:59:18,345 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_oJWBt4hwO9PwfGOyZ8H3CfAK/runs "HTTP/1.1 200 OK"
2024-07-03 19:59:18,599 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_oJWBt4hwO9PwfGOyZ8H3CfAK/runs/run_H69x6DxzTn03SaBNidVYRPxc "HTTP/1.1 200 OK"
2024-07-03 19:59:18,609 - INFO - Status: queued
2024-07-03 19:59:18,830 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_oJWBt4hwO9PwfGOyZ8H3CfAK/runs/run_H69x6DxzTn03SaBNidVYRPxc "HTTP/1.1 200 OK"
2024-07-03 19:59:38,833 - INFO - Status: queued
2024-07-03 19:59:39,151

Tool call: [RequiredActionFunctionToolCall(id='call_nviXa99spIO45VA5pvsXasDp', function=Function(arguments='{"reasoning": "Analyzing the landing logs reveals a structured approach to stabilizing and landing the spacecraft. First, we need to focus on stabilizing the spacecraft by minimizing its angular velocity, horizontal and vertical velocities, and ensuring it is over the landing zone. Then, we can perform a gentle landing. To accomplish this, the code should react to the current state of the spaceship, focusing on four key factors: X velocity, Y velocity, Angle, and Angular velocity.\\n\\nHere\'s a step-by-step reasoning:\\n1. **Stabilization**:\\n    - If the ship\'s angle is not close to zero, adjust it by pushing the appropriate side engine (left or right) to reduce the angular velocity.\\n    - If the X velocity is high, push the opposite side engine to counter this.\\n2. **Horizontal Positioning**:\\n    - If the ship is too far left or right (X position), push the opposite sid

c:\Users\adria\anaconda3\Lib\site-packages\gymnasium\wrappers\record_video.py:94: UserWarning: WARN: Overwriting existing videos at c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Semilla 38
Moviepy - Building video c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4\rl-video-episode-0.mp4.
Moviepy - Writing video c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4\rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4\rl-video-episode-0.mp4
Número de instantes: 117. Tamaño de logs: 69
episode  1 score -1157.908 avg score -9.982


...

Moviepy - Done !
Moviepy - video ready c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_5.mp4\rl-video-episode-0.mp4
Número de instantes: 846. Tamaño de logs: 433
episode  1 score -371.466 avg score -0.440


2024-07-03 20:05:53,779 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_oJWBt4hwO9PwfGOyZ8H3CfAK/runs/run_BKsENnhWSmT5dVK3Ea2WwbY9/submit_tool_outputs "HTTP/1.1 200 OK"
2024-07-03 20:05:53,783 - INFO - Compilación exitosa.
2024-07-03 20:05:53,784 - INFO - Status: requires_action
2024-07-03 20:05:54,046 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_oJWBt4hwO9PwfGOyZ8H3CfAK/runs/run_BKsENnhWSmT5dVK3Ea2WwbY9 "HTTP/1.1 200 OK"
2024-07-03 20:06:14,050 - INFO - Status: in_progress
2024-07-03 20:06:14,325 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_oJWBt4hwO9PwfGOyZ8H3CfAK/runs/run_BKsENnhWSmT5dVK3Ea2WwbY9 "HTTP/1.1 200 OK"
2024-07-03 20:06:34,330 - INFO - These are the logs generated by your last code: {"landing attempt": 1, "logs": [{"time": 0, "action": 1, "current status": [0.007, 1.417, 0.37, 0.131, -0.007, -0.044, 0.0, 0.0], "score": 1.387}, {"time": 1, "action": 1, "current status": [0.011, 1.42, 0.362, 0.104, -0.007, -

user: [TextContentBlock(text=Text(annotations=[], value='These are the logs generated by your last code: {"landing attempt": 1, "logs": [{"time": 0, "action": 1, "current status": [0.007, 1.417, 0.37, 0.131, -0.007, -0.044, 0.0, 0.0], "score": 1.387}, {"time": 1, "action": 1, "current status": [0.011, 1.42, 0.362, 0.104, -0.007, -0.011, 0.0, 0.0], "score": 1.282}, {"time": 2, "action": 1, "current status": [0.015, 1.421, 0.353, 0.077, -0.006, 0.026, 0.0, 0.0], "score": 1.473}, {"time": 3, "action": 1, "current status": [0.018, 1.423, 0.341, 0.051, -0.002, 0.072, 0.0, 0.0], "score": 1.809}, {"time": 4, "action": 1, "current status": [0.021, 1.423, 0.332, 0.024, 0.003, 0.11, 0.0, 0.0], "score": 1.038}, {"time": 5, "action": 1, "current status": [0.025, 1.423, 0.323, -0.003, 0.011, 0.146, 0.0, 0.0], "score": 0.221}, {"time": 6, "action": 1, "current status": [0.028, 1.422, 0.312, -0.03, 0.02, 0.189, 0.0, 0.0], "score": 0.039}, {"time": 7, "action": 1, "current status": [0.031, 1.421, 0.30

2024-07-03 20:06:35,754 - INFO - HTTP Request: DELETE https://api.openai.com/v1/assistants/asst_BEVYe7HSWmLAsK1s6KakV1M6 "HTTP/1.1 200 OK"
2024-07-03 20:06:35,978 - INFO - HTTP Request: DELETE https://api.openai.com/v1/threads/thread_oJWBt4hwO9PwfGOyZ8H3CfAK "HTTP/1.1 200 OK"
2024-07-03 20:06:35,981 - INFO - 
Ejecución finalizada.




Asistente vaciado.
Run(id='run_BKsENnhWSmT5dVK3Ea2WwbY9', assistant_id='asst_BEVYe7HSWmLAsK1s6KakV1M6', cancelled_at=None, completed_at=1720029954, created_at=1720029905, expires_at=None, failed_at=None, incomplete_details=None, instructions="Your task is: \n    1. Analyze and reason about the results received in the last landing attempts. This is the schema of the landing logs: {'description': 'Log data for each step of the spacecraft landing environment.', 'landing attempt': {'type': 'integer', 'description': 'The episode number.'}, 'logs': {'time': {'type': 'integer', 'description': 'The instant within the landing attempt where the current log is taken.'}, 'action': {'type': 'integer', 'description': 'The possible actions taken.', 'options': {'0': 'Switch off engines', '1': 'Push left engine', '2': 'Push both engines (upwards)', '3': 'Push right engine'}}, 'current status': {'type': 'array', 'description': 'The state of the environment after the action is taken.', 'positions': {'0':

#### Take a step back.
Abstraer primero el problema.

### Incluir supervisor encargado del stepback

In [14]:
def inicializar_agente(model, msg, description, instructions, name, tools):
    """ Inicializa un agente con un asistente y un hilo.
    
    Args:
        model: str: el modelo de lenguaje a utilizar
        msg: str: el mensaje inicial del hilo
        
    Returns:
        agente: AssistantOpenAI: el agente inicializado
        asistente: dict: el asistente creado
        hilo: dict: el hilo creado
    """
    
    agente = AssistantOpenAI(ARCLABKEY_OPENAI)
    # Crea un asistente
    asistente = agente.create_assistant(model=model, description=description, instructions=instructions, name=name, tools=tools)
    # Crea un hilo
    hilo = agente.create_thread()
    # Añade un mensaje inicial al hilo.
    msg = agente.add_message(hilo.id, role="user", content=msg)
    
    return agente, asistente, hilo


def run_message_assistant(agente, asistente, hilo, tool_choice="auto"):
    """ Ejecuta un mensaje en el asistente.
    
    Args:
        agente: AssistantOpenAI: el agente
        asistente: dict: el asistente
        hilo: dict: el hilo
        tool_choice: str: la elección de herramienta a utilizar
        
    Returns:
        ejecucion: dict: la ejecución del mensaje
        response: dict: la respuesta del asistente
    """
    # Ejecuta el hilo
    ejecucion = agente.run(hilo.id, asistente.id, tool_choice=tool_choice)
    response = agente.get_run(ejecucion.id, hilo.id)

            # Esperamos a que la respuesta esté lista.
    while response.status not in ["completed", "failed", "requires_action"]:
        logger.info(f"Status: {ejecucion.status}")
        response = agente.get_run(ejecucion.id, hilo.id)
        time.sleep(20)
    return ejecucion, response


In [14]:
import traceback, importlib
import Action

Action = importlib.reload(Action)

def create_and_run_llm_loop(Incial_msg, code_msg, logger, model="gpt-3.5-turbo", num_iterations=5):
    
    # Creamos el asistente.
    #agente, asistente, hilo = inicializar_agente(model, Incial_msg, DESCRIPTION, INSTRUCTIONS, NAME, TOOLS)
    agente = AssistantOpenAI(ARCLABKEY_OPENAI)
    # Crea un asistente
    asistente = agente.create_assistant(model=model, description=DESCRIPTION, instructions=INSTRUCTIONS, name=NAME, tools=TOOLS)
    # Crea un hilo
    hilo = agente.create_thread()
    # Añade un mensaje inicial al hilo.
    msg = agente.add_message(hilo.id, role="user", content=Incial_msg)
    
    # Primero ejecutamos el razonamiento de STEP BACK.
    #ejecucion, response = run_message_assistant(agent, asistente, hilo, tool_choice='none')
    # Ejecuta el hilo
    ejecucion = agente.run(hilo.id, asistente.id, tool_choice='none')
    response = agente.get_run(ejecucion.id, hilo.id)

            # Esperamos a que la respuesta esté lista.
    while response.status not in ["completed", "failed", "requires_action"]:
        logger.info(f"Status: {ejecucion.status}")
        response = agente.get_run(ejecucion.id, hilo.id)
        print("Waiting for response...")
        time.sleep(20)
    
    #print(response)
    msgs = agente.mostrar_mensajes(hilo.id)
    print(msgs[0].data[0].content[0].text.value)
    
    # Añadimos el mensaje de generación de código.
    msg = agente.add_message(hilo.id, role="user", content=code_msg)
    
    # Bucle de aprendizaje del asistente.
    for i in range(num_iterations):
        
        logger.info(f"Iteration: {i+1}")
        compiled = False

        # Si en la presente iteración no se ha compilado el código, se pide al asistente corregir los errores antes de iterar.
        while not compiled:
            # Ejecuta el hilo
            ejecucion = agente.run(hilo.id, asistente.id, tool_choice='required')
            response = agente.get_run(ejecucion.id, hilo.id)

            # Descomponemos los elementos de la respuesta.
            logger.info(f"Status: {response.status}")
            if response.status == "completed":
                return response
            print(response)
            tool_call = response.required_action.submit_tool_outputs.tool_calls
            print(f"Tool call: {tool_call}")

            # Convierte el string a un diccionario
            code_dict = json.loads(tool_call[0].function.arguments)
            logger.info(f"Arguments: {code_dict}")
            
            # Obtén el código Python de la llamada a la herramienta
            code = code_dict["code"]
            filename = code_dict["filename"]  

            logger.info(f"\nCodigo generado:\n{code}")

            # Ejecuta el código Python
            try:
                store_code_in_file(code, filename)
                time.sleep(1) # Pequeño retraso para que el sistema operativo pueda reflejar los cambios en el archivo
                
                # Hacemos commit de los cambios en el repositorio para analizar las modificaciones del agente.
                commit_changes(r"C:\Users\adria\Repos\TFM__LLM_landing_self-refinement", f"Stepback 4o. Iteración {i+1}.") 
                
                importlib.reload(Action) # Recargamos el módulo de acciones para que se actualice con las modificaciones del agente.

                # Ejecutamos el código generado.
                logs = lunar_lander(n_games=1, display=True, recoder=True, video_filename=f"video/iteration_{i+1}.mp4")
                
                # Devolvemos la respuesta al asistente.
                for call in tool_call:
                    agente.devolver_respuesta(response.id, hilo.id, tool_outputs=[{"tool_call_id": call.id, "output": "Run successful."}])            
                compiled = True
                logger.info(f"Compilación exitosa.")
                
                # Esperamos a que el agente esté listo para recibir mensajes y le añadimos el resultado de la iteración.
                while response.status not in ["completed", "failed", "expired"]:
                    logger.info(f"Status: {response.status}")
                    response = agente.get_run(response.id, hilo.id)
                    time.sleep(20)
                
                msg = f"""These are the logs generated by your last code: {logs}. Analyze the performance of the spacecraft and how it differs from the desired result. Be guided by the scoring system. Identify the cause of errors in your code and modify it without fear of making major changes. Think deeply about the priorities of your code and how to order and combine them correctly to achieve success."""
                logger.info(msg)    
                agente.add_message(hilo.id, role="user", content=msg)
            
            # Alimentamos el asistente con el error generado en la ejecución del código.    
            except Exception as e:
                logger.exception("Error: %s", e)
                error_trace = traceback.format_exc()
                for call in tool_call:
                    agente.devolver_respuesta(response.id, hilo.id, tool_outputs=[{"tool_call_id": call.id, "output": "ERROR."}]) 
                logger.error(f"Error: {e}.")
                while response.status not in ["completed", "failed", "expired"]:
                    logger.info(f"Status: {response.status}")
                    response = agente.get_run(ejecucion.id, hilo.id)
                    time.sleep(30)
                msg = f"The code generated has an error. Please, try again. Error: {e}. Trace: {error_trace}"   
                logger.error(msg)   
                agente.add_message(hilo.id, role="assistant", content=msg)
    
    agente.mostrar_mensajes(hilo.id)  
    vaciar_agente(agente)
    
    logger.info("\nEjecución finalizada.\n\n")       

    return response


def vaciar_agente(agente):
    for assistant in agente.assistants:
        agente.delete_assistant(assistant)

    for thread in agente.threads:
        agente.delete_thread(thread)
        
    print("Asistente vaciado.")


In [15]:
DESCRIPTION = "You are an expert spacecraft landing agent. Your specialty is to take the appropriate action at each instant of time based on the environment and state of the spacecraft. To achieve this, you put your knowledge base and the knowledge you acquire by analyzing each execution into a python function in charge of directing the landing."
INSTRUCTIONS = f"""To complete the task you must follow the following steps and indications:
1. Analyze and reason about the records received in the last landing attempts to learn how the spacecraft behaves in the environment. Keep in mind that conditions may vary.
2. To measure how good a landing is, a scoring system that appears in the logs is used, your role is to maximize it. To do so, keep the following in mind:
    - Increases/decreases the closer/further the spacecraft is from the landing area (both axes).
    - Increased/decreased the slower/faster the spacecraft is moving.
    - Decreased the more the spacecraft is tilted.
    - Decreased by 0.03 points each frame a side engine is firing and 0.3 points for the center engine.
    - Receive 100 points for a successful landing and lose them for crashing.
3. IMPORTANT. Use the following tips in your reasoning to achieve a successful landing:
    - First you have to stabilize the falling ship (speed, angle and location), keep falling under control and then land gently at the end.
    - Use all the elements of the vector of observations, all are relevant to make the right decision.
    - Pay close attention to successful events.
    - Environment and ship conditions may change but your code must be effective for all cases.
    - Find a balance in your landing policy to maximize your score (must exceed 200 points).
    - If you are rotating to the left (negative values), you should fire the left engine and vice versa. But always prioritizing a controlled speed with the main engine.
4. Add your decision code to the 'act' method and locate errors in it if the landing is unsuccessful.
5. Save the code of the act function in the file 'Action.py' using store_code_in_file function.
"""
# 6. Improve your results and correct also any programming error you may have generated in your last code if they exist.
NAME = "Spacecraft Landing Decision coder"


initial_msg = f"This is the record of an example of a successful landing in this environment, but under other conditions: {success_logs}. This is the code of the initial function: {agent_initial_code} and these are the execution logs of one landing attempt: {initial_logs}. Take a deep breath and reason step-by-step.What does each value and each change in the observations mean? What effects does each action have on the current state observations? What are the steps to follow to achieve a successful landing? Do not implement any code."
code_msg = f"After the reasoning with the processes behind the decision making, code very carefully when structuring your code, keep in mind that certain actions can be left with starvation. Save the code of the act function in the file 'Action.py' using store_code_in_file function tool, function calling is required."



In [16]:
logger = configura_log('Spacecraft_4o_Stepback.log')
response = create_and_run_llm_loop(initial_msg, code_msg, logger, model="gpt-4o", num_iterations=5)

print(response)

2024-07-01 20:24:30,902 - INFO - HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
2024-07-01 20:24:31,107 - INFO - HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"
2024-07-01 20:24:31,353 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_Z2XFcAqd7BpvTGXwLKXUdXLi/messages "HTTP/1.1 200 OK"
2024-07-01 20:24:31,917 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_Z2XFcAqd7BpvTGXwLKXUdXLi/runs "HTTP/1.1 200 OK"
2024-07-01 20:24:32,130 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_Z2XFcAqd7BpvTGXwLKXUdXLi/runs/run_rBlWNoPJc2HIQNThxrjQvaiT "HTTP/1.1 200 OK"
2024-07-01 20:24:32,132 - INFO - Status: queued
2024-07-01 20:24:32,336 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_Z2XFcAqd7BpvTGXwLKXUdXLi/runs/run_rBlWNoPJc2HIQNThxrjQvaiT "HTTP/1.1 200 OK"


Waiting for response...


In [ ]:
agente = AssistantOpenAI(ARCLABKEY_OPENAI)

# Crea un asistente
asistente = agente.create_assistant(model="gpt-3.5-turbo", description=DESCRIPTION, instructions=INSTRUCTIONS, name=NAME, tools=TOOLS)
    
# Crea un hilo
hilo = agente.create_thread()

initial_msg = f"This is the record of an example of a successful landing in this environment, but under other conditions: {success_logs}. This is the code of the initial function: {agent_initial_code} and these are the execution logs of one landing attempt: {initial_logs}. Before coding take a deep breath and reason step-by-step.What does each value and each change in the observations mean? What effects does each action have on the current state observations? What are the steps to follow to achieve a successful landing?"
    
# Añade un mensaje inicial al hilo.
msg = agente.add_message(hilo.id, role="user", content=initial_msg)


# Ejecuta el hilo
ejecucion = agente.run(hilo.id, asistente.id, tool_choice='none')
response = agente.get_run(ejecucion.id, hilo.id)

# Esperamos a que la respuesta esté lista.
while response.status not in ["completed", "failed", "requires_action"]:
    #logger.info(f"Status: {ejecucion.status}")
    print("Waiting for response...")
    response = agente.get_run(ejecucion.id, hilo.id)
    time.sleep(20)
print(response)


agente.mostrar_mensajes(hilo.id)

Waiting for response...
Waiting for response...
Run(id='run_5ip2qIKcf1KIU1L1XCyvPVyK', assistant_id='asst_qqONwHALF6YCJlMyXttFXsnu', cancelled_at=None, completed_at=1719491913, created_at=1719491908, expires_at=None, failed_at=None, incomplete_details=None, instructions="To complete the task you must follow the following steps and indications:\n1. Analyze and reason about the records received in the last landing attempts to learn how the spacecraft behaves in the environment. Keep in mind that conditions may vary.\n2. To measure how good a landing is, a scoring system that appears in the logs is used, your role is to maximize it. To do so, keep the following in mind:\n    - Increases/decreases the closer/further the spacecraft is from the landing area (both axes).\n    - Increased/decreased the slower/faster the spacecraft is moving.\n    - Decreased the more the spacecraft is tilted.\n    - Decreased by 0.03 points each frame a side engine is firing and 0.3 points for the center engin

(SyncCursorPage[Message](data=[Message(id='msg_4CYNsLrcWXj8sEnvIm9EyV4G', assistant_id='asst_qqONwHALF6YCJlMyXttFXsnu', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Based on the successful landing record and the scoring system provided, we can infer some strategies to achieve a successful landing with a high score:\n\n1. Initially, the spacecraft should start with a gentle descent while maintaining a controlled speed. This helps in stabilizing the falling ship.\n2. As the spacecraft approaches the landing area, it should decrease the altitude gradually and adjust the horizontal position to align with the landing location.\n3. The spacecraft should minimize lateral movement and keep the speed under control to ensure a safe landing.\n4. Utilize the main engine for controlled descent and adjust the tilt angle to reduce lateral velocity.\n5. When close to the landing area, reduce the speed further and ensure the spacecraft is aligned properl

In [19]:
msgs, ids = agente.mostrar_mensajes(hilo.id)
print(msgs)
print(msgs.data[0].content[0].text.value)

assistant: [TextContentBlock(text=Text(annotations=[], value="Based on the successful landing record and the scoring system provided, we can infer some strategies to achieve a successful landing with a high score:\n\n1. Initially, the spacecraft should start with a gentle descent while maintaining a controlled speed. This helps in stabilizing the falling ship.\n2. As the spacecraft approaches the landing area, it should decrease the altitude gradually and adjust the horizontal position to align with the landing location.\n3. The spacecraft should minimize lateral movement and keep the speed under control to ensure a safe landing.\n4. Utilize the main engine for controlled descent and adjust the tilt angle to reduce lateral velocity.\n5. When close to the landing area, reduce the speed further and ensure the spacecraft is aligned properly for a gentle landing.\n6. Avoid unnecessary firing of the side engines as they decrease the overall score. Only use them for small adjustments if nece

TypeError: 'SyncCursorPage[Message]' object is not subscriptable

### Dos agentes

In [15]:
import traceback, importlib
import Action

Action = importlib.reload(Action)

def create_and_run_llm_loop(Incial_msg, code_msg, logger, model="gpt-3.5-turbo", num_iterations=5):
    
    # Creamos el asistente.
    #agente, asistente, hilo = inicializar_agente(model, Incial_msg, DESCRIPTION, INSTRUCTIONS, NAME, TOOLS)
    agente = AssistantOpenAI(ARCLABKEY_OPENAI)
    # Crea un asistente supervisor y codificador.
    codificador = agente.create_assistant(model=model, description=DESCRIPTION, instructions=INSTRUCTIONS, name=NAME, tools=TOOLS)
    supervisor = agente.create_assistant(model=model, description=DESCRIPTION_SUPERVISOR, instructions=INSTRUCTIONS_SUPERVISOR, name=NAME_SUPERVISOR)
    # Crea un hilo
    hilo_supervisor = agente.create_thread()
    hilo_codificador = agente.create_thread()
    
    # Añade un mensaje inicial al hilo.
    msg = agente.add_message(hilo_supervisor.id, role="user", content=Incial_msg)
    
    # Primero ejecutamos el razonamiento de STEP BACK.
    #ejecucion, response = run_message_assistant(agent, asistente, hilo, tool_choice='none')
    # Ejecuta el hilo
    ejecucion = agente.run(hilo_supervisor.id, supervisor.id, tool_choice='none')
    response = agente.get_run(ejecucion.id, hilo_supervisor.id)

            # Esperamos a que la respuesta esté lista.
    while response.status not in ["completed", "failed", "requires_action"]:
        logger.info(f"Status: {ejecucion.status}")
        response = agente.get_run(ejecucion.id, hilo_supervisor.id)
        print("Waiting for response...")
        time.sleep(20)
    
    #print(response)
    msgs = agente.mostrar_mensajes(hilo_supervisor.id)
    logger.info(msgs[0].data[0].content[0].text.value)
    
    # Añadimos el mensaje de generación de código.
    msg = code_msg +  "\nSupervisor feedback: " + msgs[0].data[0].content[0].text.value
    print(msg)
    msg = agente.add_message(hilo_codificador.id, role="user", content=msg)
    
    # Bucle de aprendizaje del asistente.
    for i in range(num_iterations):
        
        logger.info(f"Iteration: {i+1}")
        compiled = False

        # Si en la presente iteración no se ha compilado el código, se pide al asistente corregir los errores antes de iterar.
        while not compiled:
            # Ejecuta el hilo
            ejecucion = agente.run(hilo_codificador.id, codificador.id)
            response = agente.get_run(ejecucion.id, hilo_codificador.id)

            # Descomponemos los elementos de la respuesta.
            logger.info(f"Status: {response.status}")
            if response.status == "completed":
                return response
            print(response)
            tool_call = response.required_action.submit_tool_outputs.tool_calls
            print(f"Tool call: {tool_call}")

            # Convierte el string a un diccionario
            code_dict = json.loads(tool_call[0].function.arguments)
            logger.info(f"Arguments: {code_dict}")
            
            # Obtén el código Python de la llamada a la herramienta
            code = code_dict["code"]
            filename = code_dict["filename"]  

            logger.info(f"\nCodigo generado:\n{code}")

            # Ejecuta el código Python
            try:
                store_code_in_file(code, filename)
                time.sleep(1) # Pequeño retraso para que el sistema operativo pueda reflejar los cambios en el archivo
                
                # Hacemos commit de los cambios en el repositorio para analizar las modificaciones del agente.
                commit_changes(r"C:\Users\adria\Repos\TFM__LLM_landing_self-refinement", f"Stepback 4o. Iteración {i+1}.") 
                
                importlib.reload(Action) # Recargamos el módulo de acciones para que se actualice con las modificaciones del agente.

                # Ejecutamos el código generado.
                logs = lunar_lander(n_games=1, display=True, recoder=True, video_filename=f"video/iteration_{i+1}.mp4")
                
                # Devolvemos la respuesta al asistente.
                for call in tool_call:
                    agente.devolver_respuesta(response.id, hilo_codificador.id, tool_outputs=[{"tool_call_id": call.id, "output": "Run successful."}])            
                compiled = True
                logger.info(f"Compilación exitosa.")
                
                # Esperamos a que el agente esté listo para recibir mensajes y le añadimos el resultado de la iteración.
                while response.status not in ["completed", "failed", "expired"]:
                    logger.info(f"Status: {response.status}")
                    response = agente.get_run(response.id, hilo_codificador.id)
                    time.sleep(20)
                
                msg = f"""These are the logs generated by your last code: {logs}. Analyze the performance of the spacecraft and how it differs from the desired result. Be guided by the scoring system. Identify the cause of errors in your code and modify it without fear of making major changes. Think deeply about the priorities of your code and how to order and combine them correctly to achieve success."""
                logger.info(msg)    
                agente.add_message(hilo_codificador.id, role="user", content=msg)
            
            # Alimentamos el asistente con el error generado en la ejecución del código.    
            except Exception as e:
                logger.exception("Error: %s", e)
                error_trace = traceback.format_exc()
                for call in tool_call:
                    agente.devolver_respuesta(response.id, hilo_codificador.id, tool_outputs=[{"tool_call_id": call.id, "output": "ERROR."}]) 
                logger.error(f"Error: {e}.")
                while response.status not in ["completed", "failed", "expired"]:
                    logger.info(f"Status: {response.status}")
                    response = agente.get_run(ejecucion.id, hilo_codificador.id)
                    time.sleep(30)
                msg = f"The code generated has an error. Please, try again. Error: {e}. Trace: {error_trace}"   
                logger.error(msg)   
                agente.add_message(hilo_codificador.id, role="assistant", content=msg)
    
    agente.mostrar_mensajes(hilo_codificador.id)  
    vaciar_agente(agente)
    
    logger.info("\nEjecución finalizada.\n\n")       

    return response


def vaciar_agente(agente):
    for assistant in agente.assistants:
        agente.delete_assistant(assistant)

    for thread in agente.threads:
        agente.delete_thread(thread)
        
    print("Asistente vaciado.")


In [16]:
DESCRIPTION = "You are an expert agent in spacecraft landing and decision making codificiation. Based on the landing records received and the indications received by a supervising agent, you must generate the code to land the spacecraft in the environment whatever the initial conditions are."
DESCRIPTION_SUPERVISOR = "You are an expert agent in spacecraft landing environments. Your mission is to analyze landing logs and the code generated by the agent specialized in coding, to guide him in improving the performance of his decision making code."
INSTRUCTIONS = f"""To complete the task you must follow the following steps and indications:
1. Analyze and reason about the records received in the last landing attempts to learn how the spacecraft behaves in the environment. Keep in mind that conditions may vary.
2. To measure how good a landing is, a scoring system that appears in the logs is used, your role is to maximize it. To do so, keep the following in mind:
    - Increases/decreases the closer/further the spacecraft is from the landing area (both axes).
    - Increased/decreased the slower/faster the spacecraft is moving.
    - Decreased the more the spacecraft is tilted.
    - Decreased by 0.03 points each frame a side engine is firing and 0.3 points for the center engine.
    - Receive 100 points for a successful landing and lose them for crashing.
    - A succesful landing must exceed 200 points.
3. This is the log schema:\n {log_description}
4. Add your decision code to the 'act' method and locate errors in it if the landing is unsuccessful.
5. Save the code of the act function in the file 'Action.py' using store_code_in_file function.
"""
INSTRUCTIONS_SUPERVISOR = f"""To complete the task you must follow the following steps and indications:
1. Revisa aterrizajes exitosos y fracasados para analizar el efecto de las acciones en cada una de las observaciones y decidir que decisión tomar en cada momento.
2. To measure how good a landing is, a scoring system that appears in the logs is used, your role is to maximize it. To do so, keep the following in mind:
    - Increases/decreases the closer/further the spacecraft is from the landing area (both axes).
    - Increased/decreased the slower/faster the spacecraft is moving.
    - Decreased the more the spacecraft is tilted.
    - Decreased by 0.03 points each frame a side engine is firing and 0.3 points for the center engine.
    - Receive 100 points for a successful landing and lose them for crashing.
    - A succesful landing must exceed 200 points.
3. This is the log schema:\n {log_description}
4. All the knowledge obtained must be expressed in the best possible way to the coding LLM agent so that it can improve the performance of the generated decision making code.
"""
# 6. Improve your results and correct also any programming error you may have generated in your last code if they exist.
NAME = "Spacecraft Landing Decision coder"
NAME_SUPERVISOR = "Spacecraft Landing Supervisor"


supervisor_msg = f"This is the record of an example of a successful landing in this environment, but under other conditions: {success_logs}. This is the code of the initial function: {agent_initial_code} and these are the execution logs of one landing attempt: {initial_logs}. Take a deep breath and reason step-by-step.What does each value and each change in the observations mean? What effects does each action have on the current state observations? What are the steps to follow to achieve a successful landing? Do not implement any code."
code_msg = f"This is the record of an example of a successful landing in this environment, but under other conditions: {success_logs}. This is the code of the initial function: {agent_initial_code} and these are the execution logs of one landing attempt: {initial_logs}. After the reasoning with the processes behind the decision making, code very carefully when structuring your code, keep in mind that certain actions can be left with starvation. Save the code of the act function in the file 'Action.py' using store_code_in_file function tool, function calling is required."



In [17]:
logger = configura_log('Spacecraft_4o_Stepback.log')
response = create_and_run_llm_loop(supervisor_msg, code_msg, logger, model="gpt-4o", num_iterations=5)

print(response)

2024-07-03 20:13:38,787 - INFO - HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
2024-07-03 20:13:39,055 - INFO - HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
2024-07-03 20:13:39,320 - INFO - HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"
2024-07-03 20:13:39,541 - INFO - HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"
2024-07-03 20:13:39,833 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_PViG42Y34iK8TOuGG4SDc7qu/messages "HTTP/1.1 200 OK"
2024-07-03 20:13:40,506 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_PViG42Y34iK8TOuGG4SDc7qu/runs "HTTP/1.1 200 OK"
2024-07-03 20:13:40,738 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_PViG42Y34iK8TOuGG4SDc7qu/runs/run_EMBnqChalCCiYeU88LPwgzd3 "HTTP/1.1 200 OK"
2024-07-03 20:13:40,740 - INFO - Status: queued
2024-07-03 20:13:40,971 - INFO - HTTP Request: GET https://api.openai.com/v

Waiting for response...


2024-07-03 20:14:00,973 - INFO - Status: queued
2024-07-03 20:14:01,271 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_PViG42Y34iK8TOuGG4SDc7qu/runs/run_EMBnqChalCCiYeU88LPwgzd3 "HTTP/1.1 200 OK"


Waiting for response...


2024-07-03 20:14:21,576 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_PViG42Y34iK8TOuGG4SDc7qu/messages "HTTP/1.1 200 OK"
2024-07-03 20:14:21,818 - INFO - To achieve a successful landing in the spacecraft landing environment, it's essential to understand each observation's values and the effects of each action. Here's a detailed breakdown:

### Observation Values Explained:

1. **X Position:** 
   - The horizontal position of the spacecraft relative to the landing zone.
   - Range: [-1.5, 1.5]

2. **Y Position:**
   - The vertical position of the spacecraft.
   - Range: [-1.5, 1.5]

3. **X Velocity:**
   - The horizontal velocity of the spacecraft.
   - Range: [-5.0, 5.0]

4. **Y Velocity:**
   - The vertical velocity of the spacecraft.
   - Range: [-5.0, 5.0]

5. **Angle:**
   - The angle of the spacecraft relative to vertical.
   - Range: [-3.14, 3.14] (in radians, negative for left, positive for right)

6. **Angular Velocity:**
   - The rate of change of the an

assistant: [TextContentBlock(text=Text(annotations=[], value="To achieve a successful landing in the spacecraft landing environment, it's essential to understand each observation's values and the effects of each action. Here's a detailed breakdown:\n\n### Observation Values Explained:\n\n1. **X Position:** \n   - The horizontal position of the spacecraft relative to the landing zone.\n   - Range: [-1.5, 1.5]\n\n2. **Y Position:**\n   - The vertical position of the spacecraft.\n   - Range: [-1.5, 1.5]\n\n3. **X Velocity:**\n   - The horizontal velocity of the spacecraft.\n   - Range: [-5.0, 5.0]\n\n4. **Y Velocity:**\n   - The vertical velocity of the spacecraft.\n   - Range: [-5.0, 5.0]\n\n5. **Angle:**\n   - The angle of the spacecraft relative to vertical.\n   - Range: [-3.14, 3.14] (in radians, negative for left, positive for right)\n\n6. **Angular Velocity:**\n   - The rate of change of the angle.\n   - Range: [-5.0, 5.0]\n\n7. **Left Contact Sensor:**\n   - Indicates whether the l

2024-07-03 20:14:22,079 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_zTBYfsYvK9fZkvfeXS0HuSxj/messages "HTTP/1.1 200 OK"
2024-07-03 20:14:22,331 - INFO - Iteration: 1
2024-07-03 20:14:22,783 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_zTBYfsYvK9fZkvfeXS0HuSxj/runs "HTTP/1.1 200 OK"
2024-07-03 20:14:22,997 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_zTBYfsYvK9fZkvfeXS0HuSxj/runs/run_PZ9Ujmr3s8AmLYHK1k13iWrQ "HTTP/1.1 200 OK"
2024-07-03 20:14:23,000 - INFO - Status: in_progress


Run(id='run_PZ9Ujmr3s8AmLYHK1k13iWrQ', assistant_id='asst_HjIUSni3KRFfV0KDx36Hcajr', cancelled_at=None, completed_at=None, created_at=1720030462, expires_at=1720031062, failed_at=None, incomplete_details=None, instructions="To complete the task you must follow the following steps and indications:\n1. Analyze and reason about the records received in the last landing attempts to learn how the spacecraft behaves in the environment. Keep in mind that conditions may vary.\n2. To measure how good a landing is, a scoring system that appears in the logs is used, your role is to maximize it. To do so, keep the following in mind:\n    - Increases/decreases the closer/further the spacecraft is from the landing area (both axes).\n    - Increased/decreased the slower/faster the spacecraft is moving.\n    - Decreased the more the spacecraft is tilted.\n    - Decreased by 0.03 points each frame a side engine is firing and 0.3 points for the center engine.\n    - Receive 100 points for a successful la

AttributeError: 'NoneType' object has no attribute 'submit_tool_outputs'

### JSON MODE

In [15]:
import traceback, importlib
import Action

Action = importlib.reload(Action)

def create_and_run_llm_loop(Incial_msg, logger, model="gpt-3.5-turbo", num_iterations=10):
    
    agente = AssistantOpenAI(ARCLABKEY_OPENAI)

    # Crea un asistente
    asistente = agente.create_assistant_json_mode(model=model, description=DESCRIPTION, instructions=INSTRUCTIONS, name=NAME)
    
    # Crea un hilo
    hilo = agente.create_thread()
    
    # Añade un mensaje inicial al hilo.
    msg = agente.add_message(hilo.id, role="user", content=Incial_msg)

    # Bucle de aprendizaje del asistente.
    for i in range(num_iterations):
        
        logger.info(f"Iteration: {i+1}")
        compiled = False

        # Si en la presente iteración no se ha compilado el código, se pide al asistente corregir los errores antes de iterar.
        while not compiled:
            # Ejecuta el hilo
            ejecucion = agente.run(hilo.id, asistente.id)
            response = agente.get_run(ejecucion.id, hilo.id)

            # Esperamos a que la respuesta esté lista.
            while response.status not in ["completed", "failed", "requires_action"]:
                logger.info(f"Status: {ejecucion.status}")
                response = agente.get_run(ejecucion.id, hilo.id)
                time.sleep(10)
            
            print(response.response_format)
            msgs = agente.mostrar_mensajes(hilo.id)
            msg = msgs[0].data[0].content[0].text.value
            logger.info(msg)

            # # Descomponemos los elementos de la respuesta.
            # logger.info(f"Status: {response.status}")
            # if response.status == "completed":
            #     return response
            # tool_call = response.required_action.submit_tool_outputs.tool_calls
            # print(f"Tool call: {tool_call}")

            # Convierte el string a un diccionario
            code_dict = json.loads(msg)
            logger.info(f"Arguments: {code_dict}")
            
            # Obtén el código Python de la llamada a la herramienta
            code = code_dict["code"]
            filename = code_dict["filename"]  
            reasoning = code_dict["reasoning"]

            logger.info(f"\nRazonamiento:\n{reasoning}")
            logger.info(f"\nCodigo generado:\n{code}")

            # Ejecuta el código Python
            try:
                store_code_in_file(reasoning, code, filename)
                time.sleep(1) # Pequeño retraso para que el sistema operativo pueda reflejar los cambios en el archivo
                
                # Hacemos commit de los cambios en el repositorio para analizar las modificaciones del agente.
                commit_changes(r"C:\Users\adria\Repos\TFM__LLM_landing_self-refinement", f"4o JSONMODE. Iteración {i+1}.") 
                
                importlib.reload(Action) # Recargamos el módulo de acciones para que se actualice con las modificaciones del agente.

                # Ejecutamos el código generado.
                logs = lunar_lander(n_games=1, display=True, recoder=True, video_filename=f"video/iteration_{i+1}.mp4")
                
                # # Devolvemos la respuesta al asistente.
                # for call in tool_call:
                #     agente.devolver_respuesta(response.id, hilo.id, tool_outputs=[{"tool_call_id": call.id, "output": "Run successful."}])            
                compiled = True
                logger.info(f"Compilación exitosa.")
                
                # Esperamos a que el agente esté listo para recibir mensajes y le añadimos el resultado de la iteración.
                # while response.status not in ["completed", "failed", "expired"]:
                #     logger.info(f"Status: {response.status}")
                #     response = agente.get_run(response.id, hilo.id)
                #     time.sleep(20)
                
                msg = f"""These are the logs generated by your last code: {logs}. Analyze the performance of the spacecraft and how it differs from the desired result. Be guided by the scoring system. Identify the cause of errors in your code and modify it without fear of making major changes. Think deeply about the priorities of your code and how to order and combine them correctly to achieve success. Remember that the code is a reflection of your reasoning and the way you approach the problem."""
                logger.info(msg)    
                agente.add_message(hilo.id, role="user", content=msg)
            
            # Alimentamos el asistente con el error generado en la ejecución del código.    
            except Exception as e:
                logger.exception("Error: %s", e)
                error_trace = traceback.format_exc()
                # for call in tool_call:
                #     agente.devolver_respuesta(response.id, hilo.id, tool_outputs=[{"tool_call_id": call.id, "output": "ERROR."}]) 
                # logger.error(f"Error: {e}.")
                while response.status not in ["completed", "failed", "expired"]:
                    logger.info(f"Status: {response.status}")
                    response = agente.get_run(ejecucion.id, hilo.id)
                    time.sleep(30)
                msg = f"The code generated has an error. Please, try again. Error: {e}. Trace: {error_trace}"   
                logger.error(msg)   
                agente.add_message(hilo.id, role="assistant", content=msg)
    
    agente.mostrar_mensajes(hilo.id)  
    vaciar_agente(agente)
    
    logger.info("\nEjecución finalizada.\n\n")       

    return response


def vaciar_agente(agente):
    for assistant in agente.assistants:
        agente.delete_assistant(assistant)

    for thread in agente.threads:
        agente.delete_thread(thread)
        
    print("Asistente vaciado.")

In [21]:
DESCRIPTION = "You are an expert spacecraft landing agent. Your specialty is to take the appropriate action at each instant of time based on the environment and state of the spacecraft. To achieve this, you put your knowledge base and the knowledge you acquire by analyzing each execution into a python function in charge of directing the landing."
INSTRUCTIONS = f"""To complete the task you must follow the following steps and indications:
1. Analyze and reason about the records received in the last landing attempts to learn how the spacecraft behaves in the environment. Keep in mind that conditions may vary.
2. To measure how good a landing is, a scoring system that appears in the logs is used, your role is to maximize it. To do so, keep the following in mind:
    - Increases/decreases the closer/further the spacecraft is from the landing area (both axes).
    - Increased/decreased the slower/faster the spacecraft is moving.
    - Decreased the more the spacecraft is tilted.
    - Decreased by 0.03 points each frame a side engine is firing and 0.3 points for the center engine.
    - Receive 100 points for a successful landing and lose them for crashing.
    - A succesful landing must exceed 200 points.
3. This is the log schema:\n {log_description}
4. Add your decision code to the 'act' method and locate errors in it if the landing is unsuccessful.
5. Provide output in valid JSON format. The data schema should be as follows:  {json.dumps(json_shcema_response)}.
6. Generate the code of the act function in the file 'Action.py'.
"""

# 6. Improve your results and correct also any programming error you may have generated in your last code if they exist.
NAME = "Spacecraft Landing Master"

initial_msg = "This is the record of an example of a successful landing in this environment, but under other conditions: {success_logs}. This is the code of the initial function: {agent_initial_code} and these are the execution logs of one landing attempt: {initial_logs}. What does each value and each change in the observations mean? What effects does each action have on the current state observations? What are the steps to follow to achieve a successful landing?. Take a deep breath and reason step-by-step. After reasoning analyze the results, learn and make better code."

logger = configura_log('Spacecraft_4o_JsonMode.log')
response = create_and_run_llm_loop(initial_msg, logger, model="gpt-4o", num_iterations=5)

print(response)

NameError: name 'json_shcema_response' is not defined

### Genetico

#### JSON Mode

In [16]:
def init_population(agente, model, initial_msg, logger, mode='json', N=5):
    dic_population = {}
    for i in range(N):
        if mode == 'json':
            asistente = agente.create_assistant_json_mode(model=model, description=DESCRIPTION, instructions=INSTRUCTIONS, name=NAME)
        else:
            asistente = agente.create_assistant(model=model, description=DESCRIPTION, instructions=INSTRUCTIONS, name=NAME, tools=TOOLS)
        asistente = agente.create_assistant_json_mode(model=model, description=DESCRIPTION, instructions=INSTRUCTIONS, name=NAME)
        hilo = agente.create_thread()
        msg = agente.add_message(hilo.id, role="user", content=initial_msg)
        logger.info(f"Asistente {i+1} creado. ID: {asistente.id}")
        dic_population[asistente.id] = (hilo, msg)
        
    return dic_population    

In [17]:
import traceback, importlib
import Action

Action = importlib.reload(Action)

def create_and_run_llm_loop(Incial_msg, logger, model="gpt-3.5-turbo", num_iterations=10):
    
    agente = AssistantOpenAI(ARCLABKEY_OPENAI)

    # Crea un asistente
    dic_population = init_population(agente, model, Incial_msg, logger, N=3)

    # Bucle de aprendizaje del asistente.
    for i in range(num_iterations):
        
        logger.info(f"Iteration: {i+1}")
        compiled = False
        resultados = [1, float("-inf")]
        # Ejecuta los asistentes al mismo tiempo.
        for asistente_id in dic_population.keys():
            logger.info(f"Asistente: {asistente_id}")
            hilo, msg = dic_population[asistente_id]
            asistente = agente.get_assistant(asistente_id)
            compiled = False
            
            # Si en la presente iteración no se ha compilado el código, se pide al asistente corregir los errores antes de iterar.
            while not compiled:
                # Ejecuta el hilo
                ejecucion = agente.run(hilo.id, asistente.id)
                response = agente.get_run(ejecucion.id, hilo.id)

                # Esperamos a que la respuesta esté lista.
                while response.status not in ["completed", "failed", "requires_action"]:
                    logger.info(f"Status: {ejecucion.status}")
                    response = agente.get_run(ejecucion.id, hilo.id)
                    time.sleep(10)
                
                msgs = agente.mostrar_mensajes(hilo.id)
                msg = msgs[0].data[0].content[0].text.value
                #logger.info(msg)

                # Convierte el string a un diccionario
                code_dict = json.loads(msg)
                #logger.info(f"Arguments: {code_dict}")
                
                # Obtén el código Python de la llamada a la herramienta
                code = code_dict["code"]
                filename = code_dict["filename"]  
                reasoning = code_dict["reasoning"]

                logger.info(f"\nRazonamiento:\n{reasoning}")
                logger.info(f"\nCodigo generado:\n{code}")

                # Ejecuta el código Python
                try:
                    store_code_in_file(reasoning, code, filename)
                    time.sleep(1) # Pequeño retraso para que el sistema operativo pueda reflejar los cambios en el archivo
                    
                    # Hacemos commit de los cambios en el repositorio para analizar las modificaciones del agente.
                    commit_changes(r"C:\Users\adria\Repos\TFM__LLM_landing_self-refinement", f"4o genetic JSON
                                   . Iteración {i+1}.") 
                    
                    importlib.reload(Action) # Recargamos el módulo de acciones para que se actualice con las modificaciones del agente.

                    # Ejecutamos el código generado.
                    logs = lunar_lander(n_games=1, display=True, recoder=True, video_filename=f"video/Asitente_{asistente_id}/iteration_{i+1}.mp4")
                    json_logs = json.loads(logs)
                    
                    if float(json_logs['total score']) > resultados[1]:
                        resultados = [asistente_id, float(json_logs["total score"]), code]
                    resultados.append(json_logs["total score"])
                             
                    compiled = True
                    logger.info(f"Compilación exitosa.")
                    
                    msg = f"""These are the logs generated by your last code: {logs}. Analyze the performance of the spacecraft and how it differs from the desired result. Be guided by the scoring system. Identify the cause of errors in your code and modify it without fear of making major changes. Think deeply about the priorities of your code and how to order and combine them correctly to achieve success. Remember that the code is a reflection of your reasoning and the way you approach the problem."""
                    logger.info(msg)    
                    agente.add_message(hilo.id, role="user", content=msg)
                
                # Alimentamos el asistente con el error generado en la ejecución del código.    
                except Exception as e:
                    logger.exception("Error: %s", e)
                    error_trace = traceback.format_exc()

                    while response.status not in ["completed", "failed", "expired"]:
                        logger.info(f"Status: {response.status}")
                        response = agente.get_run(ejecucion.id, hilo.id)
                        time.sleep(30)
                    msg = f"The code generated has an error. Please, try again. Error: {e}. Trace: {error_trace}"   
                    logger.error(msg)   
                    agente.add_message(hilo.id, role="assistant", content=msg)
        logger.info(f"Mejor resultado: Asistentente {resultados[0]} con puntuación {resultados[1]}")
        # Después de determinar el asistente con la máxima puntuación y antes de finalizar el bucle de iteraciones
        for asistente_id in dic_population.keys():
            if asistente_id != resultados[0]:  # Verifica que no sea el asistente con la máxima puntuación
                hilo, _ = dic_population[asistente_id]
                # Aquí puedes añadir el resultado al asistente que no tiene la máxima puntuación
                # Por ejemplo, suponiendo que tienes una función para añadir mensajes o resultados:
                msg_resultado = f"The best result for the population was {resultados[1]} achieved with the following code: {resultados[2]}. \nCompare it with yours and draw conclusions. Re-generate the code step by step and think deeply to improve both results."
                agente.add_message(hilo.id, role="user", content=msg_resultado)
        
    
    agente.mostrar_mensajes(hilo.id)  
    vaciar_agente(agente)
    
    logger.info("\nEjecución finalizada.\n\n")       

    return response


def vaciar_agente(agente):
    for assistant in agente.assistants:
        agente.delete_assistant(assistant)

    for thread in agente.threads:
        agente.delete_thread(thread)
        
    print("Asistente vaciado.")

In [18]:
DESCRIPTION = "You are an expert spacecraft landing agent. Your specialty is to take the appropriate action at each instant of time based on the environment and state of the spacecraft. To achieve this, you put your knowledge base and the knowledge you acquire by analyzing each execution into a python function in charge of directing the landing."
INSTRUCTIONS = f"""To complete the task you must follow the following steps and indications:
1. Analyze and reason about the records received in the last landing attempts to learn how the spacecraft behaves in the environment. Keep in mind that conditions may vary.
2. To measure how good a landing is, a scoring system that appears in the logs is used, your role is to maximize it. To do so, keep the following in mind:
    - Increases/decreases the closer/further the spacecraft is from the landing area (both axes).
    - Increased/decreased the slower/faster the spacecraft is moving.
    - Decreased the more the spacecraft is tilted.
    - Decreased by 0.03 points each frame a side engine is firing and 0.3 points for the center engine.
    - Receive 100 points for a successful landing and lose them for crashing.
    - A succesful landing must exceed 200 points.
3. This is the log schema:\n {log_description}
4. Add your decision code to the 'act' method and locate errors in it if the landing is unsuccessful.
5. Provide output in valid JSON format. The data schema should be as follows:  {json.dumps(json_shcema_response)}.
6. Generate the code of the act function in the file 'Action.py'.
"""

# 6. Improve your results and correct also any programming error you may have generated in your last code if they exist.
NAME = "Spacecraft Landing Master"

initial_msg = f"This is the record of an example of a successful landing in this environment, but under other conditions: {success_logs}. This is the code of the initial function: {agent_initial_code} and these are the execution logs of one landing attempt: {initial_logs}. What does each value and each change in the observations mean? What effects does each action have on the current state observations? What are the steps to follow to achieve a successful landing?. Take a deep breath and reason step-by-step. After reasoning analyze the results, learn and make better code."

logger = configura_log('Spacecraft_4o_genetic.log')
response = create_and_run_llm_loop(initial_msg, logger, model="gpt-4o", num_iterations=5)

print(response)

2024-07-03 12:41:46,651 - INFO - HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
2024-07-03 12:41:46,954 - INFO - HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"
2024-07-03 12:41:47,202 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_6lXS5bEJWx9DfAyfgXLrgrLA/messages "HTTP/1.1 200 OK"
2024-07-03 12:41:47,400 - INFO - Asistente 1 creado. ID: asst_g80WvbiVksILFWSWqlrjR0oD
2024-07-03 12:41:47,675 - INFO - HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
2024-07-03 12:41:47,883 - INFO - HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"
2024-07-03 12:41:48,182 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_OyFKkBI3jwxZSukSkfxlteFr/messages "HTTP/1.1 200 OK"
2024-07-03 12:41:48,392 - INFO - Asistente 2 creado. ID: asst_1qm0ClDWY6sldqhJdjf5xafQ
2024-07-03 12:41:48,693 - INFO - HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
2024-07

assistant: [TextContentBlock(text=Text(annotations=[], value='{"reasoning": "To achieve a successful landing, the spacecraft must be guided to a safe descent. The successful landing example provided sheds light on the relationship between actions and observations: We can deduce the following influences of actions on state parameters: \\n- Action **0**: "  , \n "code":  "import numpy as np\\n\\n# Constants\\nLEFT_ENGINE_COST = 0.03\\nCENTER_ENGINE_COST = 0.3\\nRIGHT_ENGINE_COST = 0.03\\n\\n# Indices for observation for clarity\\nX_POS = 0\\nY_POS = 1\\nX_VEL = 2\\nY_VEL = 3\\nANGLE = 4\\nANG_VEL = 5\\nLEFT_CONTACT = 6\\nRIGHT_CONTACT = 7\\n\\n\\ndef act(observation):\\n    \'\'\'\\n    The function that codifies the action to be taken in each instant of time.\\n\\n    Args:\\n        observation (numpy.array):\\n            Description: The state of the environment after the action is taken.\\n            Positions: {  \\n                0: X position,\\n                1: Y position,\\

c:\Users\adria\anaconda3\Lib\site-packages\gymnasium\wrappers\record_video.py:94: UserWarning: WARN: Overwriting existing videos at c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Semilla 38
Moviepy - Building video c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4\rl-video-episode-0.mp4.
Moviepy - Writing video c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_1.mp4\rl-video-episode-0.mp4



2024-07-03 12:42:16,952 - INFO - Compilación exitosa.         
2024-07-03 12:42:16,953 - INFO - These are the logs generated by your last code: {"landing attempt": 1, "logs": [{"time": 0, "action": 2, "current status": [0.007, 1.418, 0.377, 0.172, -0.009, -0.089, 0.0, 0.0], "score": -1.45}, {"time": 1, "action": 2, "current status": [0.011, 1.422, 0.389, 0.179, -0.013, -0.077, 0.0, 0.0], "score": -2.465}, {"time": 2, "action": 2, "current status": [0.015, 1.427, 0.395, 0.2, -0.016, -0.072, 0.0, 0.0], "score": -2.498}, {"time": 3, "action": 2, "current status": [0.019, 1.432, 0.385, 0.226, -0.02, -0.083, 0.0, 0.0], "score": -1.675}, {"time": 4, "action": 2, "current status": [0.023, 1.437, 0.389, 0.229, -0.024, -0.08, 0.0, 0.0], "score": -1.691}, {"time": 5, "action": 2, "current status": [0.027, 1.443, 0.398, 0.251, -0.028, -0.074, 0.0, 0.0], "score": -3.085}, {"time": 6, "action": 2, "current status": [0.031, 1.449, 0.392, 0.264, -0.032, -0.081, 0.0, 0.0], "score": -1.511}, {"time": 7

...

Moviepy - Done !
Moviepy - video ready c:\Users\adria\Repos\TFM__LLM_landing_self-refinement\video\iteration_5.mp4\rl-video-episode-0.mp4
Número de instantes: 90. Tamaño de logs: 55
episode  1 score -233.267 avg score -2.621


2024-07-03 12:50:39,465 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_XEJdeQ1OtT6lVxpNjoNGctL8/messages "HTTP/1.1 200 OK"
2024-07-03 12:50:39,466 - INFO - Mejor resultado: Asistentente asst_1qm0ClDWY6sldqhJdjf5xafQ con puntuación -46.801
2024-07-03 12:50:39,700 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_6lXS5bEJWx9DfAyfgXLrgrLA/messages "HTTP/1.1 200 OK"
2024-07-03 12:50:39,943 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_XEJdeQ1OtT6lVxpNjoNGctL8/messages "HTTP/1.1 200 OK"
2024-07-03 12:50:40,152 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_XEJdeQ1OtT6lVxpNjoNGctL8/messages "HTTP/1.1 200 OK"


user: [TextContentBlock(text=Text(annotations=[], value='The best result for the population was -46.801 achieved with the following code: import numpy as np\n\n# Define constants for easier adjustments\nHORIZONTAL_THRESHOLD = 0.1\nVELOCITY_THRESHOLD = 0.1\nVERTICAL_VELOCITY_THRESHOLD = -0.3\nANGLE_THRESHOLD = 0.1\n\n# Indices for observation for clarity\nX_POS = 0\nY_POS = 1\nX_VEL = 2\nY_VEL = 3\nANGLE = 4\nANG_VEL = 5\nLEFT_CONTACT = 6\nRIGHT_CONTACT = 7\n\n\ndef act(observation):\n    \'\'\'\n    Function determining the required action for optimal landing.\n\n    Args:\n        observation (numpy.array):\n            - Description: State of the environment after the action is taken.\n            - Positions: {  \n                "0": X position,\n                "1": Y position,\n                "2": X velocity,\n                "3": Y velocity,\n                "4": Angle,\n                "5": Angular velocity,\n                "6": Left contact sensor,\n                "7": Righ

2024-07-03 12:50:40,811 - INFO - HTTP Request: DELETE https://api.openai.com/v1/assistants/asst_g80WvbiVksILFWSWqlrjR0oD "HTTP/1.1 200 OK"
2024-07-03 12:50:41,012 - INFO - HTTP Request: DELETE https://api.openai.com/v1/assistants/asst_1qm0ClDWY6sldqhJdjf5xafQ "HTTP/1.1 200 OK"
2024-07-03 12:50:41,239 - INFO - HTTP Request: DELETE https://api.openai.com/v1/assistants/asst_IWiXcKSkXWwTtBvguf9N1XGP "HTTP/1.1 200 OK"
2024-07-03 12:50:41,555 - INFO - HTTP Request: DELETE https://api.openai.com/v1/threads/thread_6lXS5bEJWx9DfAyfgXLrgrLA "HTTP/1.1 200 OK"
2024-07-03 12:50:41,792 - INFO - HTTP Request: DELETE https://api.openai.com/v1/threads/thread_OyFKkBI3jwxZSukSkfxlteFr "HTTP/1.1 200 OK"
2024-07-03 12:50:42,042 - INFO - HTTP Request: DELETE https://api.openai.com/v1/threads/thread_XEJdeQ1OtT6lVxpNjoNGctL8 "HTTP/1.1 200 OK"
2024-07-03 12:50:42,043 - INFO - 
Ejecución finalizada.




Asistente vaciado.
Run(id='run_nYtzuFTJVKCNgRXYRt1QBrR9', assistant_id='asst_IWiXcKSkXWwTtBvguf9N1XGP', cancelled_at=None, completed_at=1720003823, created_at=1720003804, expires_at=None, failed_at=None, incomplete_details=None, instructions='To complete the task you must follow the following steps and indications:\n1. Analyze and reason about the records received in the last landing attempts to learn how the spacecraft behaves in the environment. Keep in mind that conditions may vary.\n2. To measure how good a landing is, a scoring system that appears in the logs is used, your role is to maximize it. To do so, keep the following in mind:\n    - Increases/decreases the closer/further the spacecraft is from the landing area (both axes).\n    - Increased/decreased the slower/faster the spacecraft is moving.\n    - Decreased the more the spacecraft is tilted.\n    - Decreased by 0.03 points each frame a side engine is firing and 0.3 points for the center engine.\n    - Receive 100 points 

#### Function calling.

In [ ]:
import traceback, importlib
import Action

Action = importlib.reload(Action)

def create_and_run_llm_loop(Incial_msg, logger, model="gpt-3.5-turbo", num_iterations=10):
    
    agente = AssistantOpenAI(ARCLABKEY_OPENAI)

    # Crea un asistente
    dic_population = init_population(agente, model, Incial_msg, logger, mode='fc', N=3)

    # Bucle de aprendizaje del asistente.
    for i in range(num_iterations):
        
        logger.info(f"Iteration: {i+1}")
        compiled = False
        resultados = [1, float("-inf")]
        # Ejecuta los asistentes al mismo tiempo.
        for asistente_id in dic_population.keys():
            logger.info(f"Asistente: {asistente_id}")
            hilo, msg = dic_population[asistente_id]
            asistente = agente.get_assistant(asistente_id)
            compiled = False
            
            # Si en la presente iteración no se ha compilado el código, se pide al asistente corregir los errores antes de iterar.
            while not compiled:
                # Ejecuta el hilo
                ejecucion = agente.run(hilo.id, asistente.id)
                response = agente.get_run(ejecucion.id, hilo.id)

                # Esperamos a que la respuesta esté lista.
                while response.status not in ["completed", "failed", "requires_action"]:
                    logger.info(f"Status: {ejecucion.status}")
                    response = agente.get_run(ejecucion.id, hilo.id)
                    time.sleep(20)

                # Descomponemos los elementos de la respuesta.
                logger.info(f"Status: {response.status}")
                if response.status == "completed":
                    return response
                tool_call = response.required_action.submit_tool_outputs.tool_calls
                print(f"Tool call: {tool_call}")

                # Convierte el string a un diccionario
                code_dict = json.loads(tool_call[0].function.arguments)
                logger.info(f"Arguments: {code_dict}")
                
                # Obtén el código Python de la llamada a la herramienta
                code = code_dict["code"]
                filename = code_dict["filename"]  
                reasoning = code_dict["reasoning"]

                logger.info(f"\nRazonamiento:\n{reasoning}")
                logger.info(f"\nCodigo generado:\n{code}")

                # Ejecuta el código Python
                try:
                    store_code_in_file(reasoning, code, filename)
                    time.sleep(1) # Pequeño retraso para que el sistema operativo pueda reflejar los cambios en el archivo
                    
                    # Hacemos commit de los cambios en el repositorio para analizar las modificaciones del agente.
                    commit_changes(r"C:\Users\adria\Repos\TFM__LLM_landing_self-refinement", f"4o genetic FC. Iteración {i+1}.") 
                    
                    importlib.reload(Action) # Recargamos el módulo de acciones para que se actualice con las modificaciones del agente.

                    # Ejecutamos el código generado.
                    logs = lunar_lander(n_games=1, display=True, recoder=True, video_filename=f"video/Asitente_{asistente_id}/iteration_{i+1}.mp4")
                    json_logs = json.loads(logs)
                    
                    if float(json_logs['total score']) > resultados[1]:
                        resultados = [asistente_id, float(json_logs["total score"]), code]
                    resultados.append(json_logs["total score"])
                    
                    # Devolvemos la respuesta al asistente.
                    for call in tool_call:
                        agente.devolver_respuesta(response.id, hilo.id, tool_outputs=[{"tool_call_id": call.id, "output": "Run successful."}])            
                    compiled = True
                    logger.info(f"Compilación exitosa.")
                    
                    # Esperamos a que el agente esté listo para recibir mensajes y le añadimos el resultado de la iteración.
                    while response.status not in ["completed", "failed", "expired"]:
                        logger.info(f"Status: {response.status}")
                        response = agente.get_run(response.id, hilo.id)
                        time.sleep(20)
                             
                    compiled = True
                    logger.info(f"Compilación exitosa.")
                    
                    msg = f"""These are the logs generated by your last code: {logs}. Analyze the performance of the spacecraft and how it differs from the desired result. Be guided by the scoring system. Identify the cause of errors in your code and modify it without fear of making major changes. Think deeply about the priorities of your code and how to order and combine them correctly to achieve success. Remember that the code is a reflection of your reasoning and the way you approach the problem."""
                    logger.info(msg)    
                    agente.add_message(hilo.id, role="user", content=msg)
                
                # Alimentamos el asistente con el error generado en la ejecución del código.    
                except Exception as e:
                    logger.exception("Error: %s", e)
                    error_trace = traceback.format_exc()
                    for call in tool_call:
                        agente.devolver_respuesta(response.id, hilo.id, tool_outputs=[{"tool_call_id": call.id, "output": "ERROR."}]) 
                    logger.error(f"Error: {e}.")
                    while response.status not in ["completed", "failed", "expired"]:
                        logger.info(f"Status: {response.status}")
                        response = agente.get_run(ejecucion.id, hilo.id)
                        time.sleep(30)
                    msg = f"The code generated has an error. Please, try again. Error: {e}. Trace: {error_trace}"   
                    logger.error(msg)   
                    agente.add_message(hilo.id, role="assistant", content=msg)
        logger.info(f"Mejor resultado: Asistentente {resultados[0]} con puntuación {resultados[1]}")
        # Después de determinar el asistente con la máxima puntuación y antes de finalizar el bucle de iteraciones
        for asistente_id in dic_population.keys():
            if asistente_id != resultados[0]:  # Verifica que no sea el asistente con la máxima puntuación
                hilo, _ = dic_population[asistente_id]
                # Aquí puedes añadir el resultado al asistente que no tiene la máxima puntuación
                # Por ejemplo, suponiendo que tienes una función para añadir mensajes o resultados:
                msg_resultado = f"The best result for the population was {resultados[1]} achieved with the following code: {resultados[2]}. \nCompare it with yours and draw conclusions. Re-generate the code step by step and think deeply to improve both results."
                agente.add_message(hilo.id, role="user", content=msg_resultado)
        
    
    agente.mostrar_mensajes(hilo.id)  
    vaciar_agente(agente)
    
    logger.info("\nEjecución finalizada.\n\n")       

    return response


def vaciar_agente(agente):
    for assistant in agente.assistants:
        agente.delete_assistant(assistant)

    for thread in agente.threads:
        agente.delete_thread(thread)
        
    print("Asistente vaciado.")

In [ ]:
DESCRIPTION = "You are an expert spacecraft landing agent. Your specialty is to take the appropriate action at each instant of time based on the environment and state of the spacecraft. To achieve this, you put your knowledge base and the knowledge you acquire by analyzing each execution into a python function in charge of directing the landing."
INSTRUCTIONS = f"""To complete the task you must follow the following steps and indications:
1. Analyze and reason about the records received in the last landing attempts to learn how the spacecraft behaves in the environment. Keep in mind that conditions may vary.
2. To measure how good a landing is, a scoring system that appears in the logs is used, your role is to maximize it. To do so, keep the following in mind:
    - Increases/decreases the closer/further the spacecraft is from the landing area (both axes).
    - Increased/decreased the slower/faster the spacecraft is moving.
    - Decreased the more the spacecraft is tilted.
    - Decreased by 0.03 points each frame a side engine is firing and 0.3 points for the center engine.
    - Receive 100 points for a successful landing and lose them for crashing.
    - A succesful landing must exceed 200 points.
3. This is the log schema:\n {log_description}
4. Add your decision code to the 'act' method and locate errors in it if the landing is unsuccessful.
5. Provide output in valid JSON format. The data schema should be as follows:  {json.dumps(json_shcema_response)}.
6. Generate the code of the act function in the file 'Action.py'.
"""

# 6. Improve your results and correct also any programming error you may have generated in your last code if they exist.
NAME = "Spacecraft Landing Master"

initial_msg = f"This is the record of an example of a successful landing in this environment, but under other conditions: {success_logs}. This is the code of the initial function: {agent_initial_code} and these are the execution logs of one landing attempt: {initial_logs}. What does each value and each change in the observations mean? What effects does each action have on the current state observations? What are the steps to follow to achieve a successful landing?. Take a deep breath and reason step-by-step. After reasoning analyze the results, learn and make better code."

logger = configura_log('Spacecraft_4o_genetic_FC.log')
response = create_and_run_llm_loop(initial_msg, logger, model="gpt-3.5-turbo", num_iterations=5)

print(response)

## Tasks
- Corregir rotación.
- Cantidad de logs.
- Partir de ejemplo exitoso para afinarlo. (y generalizar)
- Incluir take step back.
- Probar con 2 modelos. Uno encargado del razonamiento y otro de codificar. (O uno que retroalimente al otro con las conclusiones obtenidas)